In [ ]:
cd /home/yuchen/pulse2percept

In [ ]:
import shapes
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pulse2percept as p2p
import skimage
from skimage.measure import label, regionprops, regionprops_table
import math
import string
from statistics import mean
from sklearn.linear_model import LinearRegression
import shapely.geometry as geom
import statsmodels.api as sm 
import pingouin as pg
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
def curve_length(curve):
    return np.sum(np.sqrt(np.sum((curve[:-1] - curve[1:])**2,axis=1)))

# Analysis

In [ ]:
# calculate shapes 
data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['12-005','51-009','52-001'],stim_class=None)

result = pd.DataFrame({})
for i in range(len(data)):
    label_img = skimage.measure.label(data['image'][i]>0)
    regions = regionprops(label_img)
    props = regionprops_table(label_img, properties=('centroid',
                                                     'orientation',
                                                     'major_axis_length',
                                                     'minor_axis_length',
                                                     'area',
                                                    'eccentricity',
                                                    'perimeter'))
    df = pd.DataFrame(props).astype('object')
    df.at[0,'centroid-0'] = df.iloc[:, 0].tolist()  # store centroid-x
    df.at[0,'centroid-1'] = df.iloc[:, 1].tolist()  # store centroid-y
    df.at[0,'orientation'] = df.iloc[:, 2].tolist()  # store orientation
    df.at[0,'major_axis_length'] = df.iloc[:, 3].tolist()  # major
    df.at[0,'minor_axis_length'] = df.iloc[:, 4].tolist()  # minor
    df.at[0,'area'] = df.iloc[:, 5].tolist()  # area
    df.at[0,'eccentricity'] = df.iloc[:, 6].tolist()  
    df.at[0,'perimeter'] = df.iloc[:, 7].tolist()  
    result = pd.concat([result, df.iloc[:1,:]],axis=0)

result = result.rename(columns={"area":"size", "orientation":"orientation_new", "eccentricity":"eccentricity_new" })
data = pd.concat([data,result.reset_index(drop=True)],axis=1)
data_copy = data 

In [ ]:
data = data_copy.copy()

data_double = data[(data['electrode2'] != str())].reset_index(drop=True)
data_single = data[(data['electrode2'] == str())].reset_index(drop=True)
data_single['stim_class'] ='SingleElectrode'
df = data_single[['subject', 'electrode1', 'freq', 'amp1']].drop_duplicates().reset_index(drop=True)
mo = 0
de = 0
lst = []
lst_size = []
lst_major = []
lst_minor = []
lst_eccentricity = []
lst_orientation = []
lst_perimeter = []

for i in range(len(df)):
    df_temp = data_single[(data_single['subject'] == df.subject[i]) & (data_single['electrode1'] == df.electrode1[i]) & (data_single['freq'] == df.freq[i]) & (data_single['amp1'] == df.amp1[i])].reset_index(drop=True)
    ratio = len(df_temp[df_temp['num_regions']>1]) /len(df_temp)
    lst.append(ratio)
    if (ratio > 0.0) & (ratio <= 0.5):
        df_temp_1 = df_temp[df_temp.num_regions == 1]
        avg_size_1 = df_temp_1[['area']].mean()['area']
        avg_orientation_1 = df_temp_1[['orientation']].mean()['orientation']
        avg_eccentricity_1 = df_temp_1[['eccentricity']].mean()['eccentricity']
        df_temp_2 = df_temp[df_temp.num_regions > 1].reset_index(drop=True)
        for item in range(len(df_temp_2)):
            phosphene_id = 0
            distance = 9999999
            for phosphene_id_temp in range(df_temp_2.num_regions[item]):
                distance_temp = (float(df_temp_2['orientation_new'][item][phosphene_id_temp]) - avg_orientation_1)**2 + (float(df_temp_2['size'][item][phosphene_id_temp]) - avg_size_1)**2 + (float(df_temp_2['eccentricity_new'][item][phosphene_id_temp]) - avg_eccentricity_1)**2
                if distance_temp < distance:
                    distance = distance_temp
                    phosphene_id = phosphene_id_temp
            df_temp_2.at[item, 'size'] = [df_temp_2['size'][item][phosphene_id]]
            df_temp_2.at[item, 'major_axis_length'] = [df_temp_2['major_axis_length'][item][phosphene_id]]
            df_temp_2.at[item, 'minor_axis_length'] = [df_temp_2['minor_axis_length'][item][phosphene_id]]
            df_temp_2.at[item, 'orientation_new'] = [df_temp_2['orientation_new'][item][phosphene_id]]
            df_temp_2.at[item, 'eccentricity_new'] = [df_temp_2['eccentricity_new'][item][phosphene_id]]
            df_temp_2.at[item, 'perimeter'] = [df_temp_2['perimeter'][item][phosphene_id]]
            
            df_temp = pd.concat([df_temp_1, df_temp_2]).reset_index(drop=True)
        
        
    count_size = 0
    count_major = 0
    count_minor = 0
    count_eccentricity = 0
    count_orientation = 0
    count_perimeter = 0
    
    count_length = len(df_temp)
    for j in range(count_length):
        count_size += df_temp['size'][j][0]
        count_major += df_temp['major_axis_length'][j][0]
        count_minor += df_temp['minor_axis_length'][j][0]
        count_eccentricity += df_temp['eccentricity_new'][j][0]
        count_orientation += df_temp['orientation_new'][j][0]
        count_perimeter += df_temp['perimeter'][j][0]
        
        
    lst_size.append(count_size/count_length)
    lst_major.append(count_major/count_length)
    lst_minor.append(count_minor/count_length)
    lst_eccentricity.append(count_eccentricity/count_length)
    lst_orientation.append(count_orientation/count_length)
    lst_perimeter.append(count_perimeter/count_length)
    
df['helper'] = lst
df['avg_size'] = lst_size
df['avg_major'] = lst_major
df['avg_minor'] = lst_minor
df['avg_eccentricity'] = lst_eccentricity
df['avg_orientation'] = lst_orientation
df['avg_perimeter'] = lst_perimeter

df = df[df['helper'] <= 0.5].reset_index(drop=True)
data_single = df

subject = ['12-005', '51-009', '52-001']
df = pd.DataFrame({})

for subj in subject:
    lst_e = []
    lst_dtf = []
    
    s2 = shapes.subject_params[subj]
    implant,model = shapes.model_from_params(s2)
    for i in string.ascii_uppercase[0:6]: 
        for j in range(1,11):
            electrode = i + str(j)
            lst_e.append(electrode)
            lst_dtf.append(math.sqrt(implant[electrode].x**2 +implant[electrode].y**2 ))
            
    df_o = pd.DataFrame(lst_e, columns=['electrode1'])
    df_o['distance_to_fovea'] = lst_dtf
    
    if subj == '12-005':
        lst_d = [0,0,0,2,7,7,0,0,5,0,
                0,0,4,11,13,15,15,3,7,4,
                0,0,15,16,17,17,19,16,9,4,
                0,0,16,19,15,17,22,25,13,10,
                0,0,8,15,14,13,17,23,14,2,
                0,0,10,15,13,12,9,11,5]
        lst_e = lst_e[:-1]
    elif subj == '51-009':
        lst_e = ['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C2','C3','C4','C5','C6','C7','C8','C9','C10', 
               'B3','B4','B5','B6','B7','B8','B9','B10', 
               'A3','A4','A5','A6','A7','A8','A9','A10']
        lst_d = [0] * len(lst_e)
    else:
        lst_e = ['F1','F2','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C3','C4','C5','C6','C7','C8','C9','C10', 
               'B1','B3','B4','B5','B6','B7','B8','B9','B10', 
               'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10']
        lst_d = [0] * len(lst_e)
    lst_s = [subj] * len(lst_e)
    
    df_o = df_o[df_o.electrode1.isin(lst_e)]
    df_o['distance_to_implant'] = lst_d
    df_o['subject'] = lst_s
    df = pd.concat([df, df_o])
data_single = data_single.merge(df, how = 'inner', on = ['electrode1','subject'])


index_lst = []
for i in range(len(data_double)):
    if not (data_single[(data_single['subject'] == data_double.subject[i]) & (data_single['electrode1'] == data_double.electrode1[i]) & (data_single['freq'] == data_double.freq[i]) & (data_single['amp1'] == data_double.amp1[i])].empty):
        if not (data_single[(data_single['subject'] == data_double.subject[i]) & (data_single['electrode1'] == data_double.electrode2[i]) & (data_single['freq'] == data_double.freq[i]) & (data_single['amp1'] == data_double.amp1[i])].empty):
            index_lst.append(i)

In [ ]:
import scipy.stats as spst
def scatter_correlation(x, y, marker='o', color='k', ax=None, autoscale=True, text = True):
    # If data are Pandas series, use their names to label the axes:
    x_label = x.name if hasattr(x, 'name') else ''
    y_label = y.name if hasattr(y, 'name') else ''
    x = np.asarray(x)
    y = np.asarray(y)
    # Ignore NaN:
    isnan = np.isnan(x) | np.isnan(y)
    x = x[~isnan]
    y = y[~isnan]
    if not np.all(x.shape == y.shape):
        raise ValueError("x and y must have the same shape")
    if len(x) < 2:
        raise ValueError("x and y must at least have 2 data points.")
    # Scatter plot the data:
    if ax is None:
        ax = plt.gca()
    ax.scatter(x, y, marker=marker, s=50, c=color, edgecolors='w', alpha=0.5)
    # Fit the regression curve:
    slope, intercept, rval, pval, _ = spst.linregress(x, y)
    def fit(x): return slope * x + intercept
    ax.plot([np.min(x), np.max(x)], [fit(np.min(x)), fit(np.max(x))], 'k--', c=color)
    # Annotate with fitting results:
    if text == True:
        pval = ("%.2e" % pval) if pval < 0.001 else ("%.03f" % pval)
        a = ax.axis()
        ax.text(a[1], a[2], "$N$=%d\n$r$=%.3f, $p$=%s" % (len(y), rval, pval),
                va='bottom', ha='right')
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    if autoscale:
        ax.autoscale(True)
    return ax

### size

In [ ]:
df_AllSub = pd.DataFrame()

condition_list = [0]  
label_list = ['SpatialSummation','MultiElectrode']
label_list = ['MultiElectrode']
for condition in condition_list:
    for label in range(1):
        data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
        data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
        lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
        lst = lst.drop_duplicates().reset_index(drop=True)

        double = []
        single_dtf = []
        single_dti = []
        lst_subject = []
        
        distance = []
        amp = []
        freq = []
        electrode = []
        between = []
        along = []
        between_2 = []
        along_2 = []
        for i in range(len(lst)):
            electrode1 = lst.electrode1[i]
            electrode2 = lst.electrode2[i]
            temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
            if not temp1.empty and not temp2.empty and not double_temp.empty:
                count = 0
                for j in range(len(double_temp)):
                    count += sum(double_temp['size'][j])
                double.append(count/len(double_temp))
                single_dtf.append((temp1.distance_to_fovea[0] + temp2.distance_to_fovea[0])/2)
                single_dti.append((temp1.distance_to_implant[0] + temp2.distance_to_implant[0])/2)
                lst_subject.append(temp1.subject[0])
                
                subject = shapes.subject_params[temp1.subject[0]]
                implant,model = shapes.model_from_params(subject)
                
                e1_x = implant[electrode1].x
                e1_y = implant[electrode1].y
                e2_x = implant[electrode2].x
                e2_y = implant[electrode2].y
                
                distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
                
                bundle = model.grow_axon_bundles()

                curve  = model.find_closest_axon(bundle, xret=e2_x, yret=e2_y)
                line = geom.LineString(curve)
                point1 = geom.Point(e1_x,e1_y)
                between1 = point1.distance(line)

                point_on_line1 = line.interpolate(line.project(point1))
                point2 = geom.Point(e2_x,e2_y)
                point_on_line2 = line.interpolate(line.project(point2))

                if point_on_line1.x < point_on_line2.x:
                    points = np.array([[point_on_line1.x,point_on_line1.y], [point_on_line2.x,point_on_line2.y]])
                else:
                    points = np.array([[point_on_line2.x,point_on_line2.y],[point_on_line1.x,point_on_line1.y]])
                curve = curve[::-1]
                start = np.searchsorted(curve[:,0], points[0,0])
                end = np.searchsorted(curve[:,0], points[1,0])
                along1 = curve_length(curve[start:end])
                
                curve  = model.find_closest_axon(bundle, xret=e1_x, yret=e1_y)
                line = geom.LineString(curve)
                point2 = geom.Point(e2_x,e2_y)
                between2 = point2.distance(line)
                
                point_on_line2 = line.interpolate(line.project(point2))
                point1 = geom.Point(e1_x,e1_y)
                point_on_line1 = line.interpolate(line.project(point1))

                if point_on_line2.x < point_on_line1.x:
                    points = np.array([[point_on_line2.x,point_on_line2.y], [point_on_line1.x,point_on_line1.y]])
                else:
                    points = np.array([[point_on_line1.x,point_on_line1.y],[point_on_line2.x,point_on_line2.y]])
                curve = curve[::-1]
                start = np.searchsorted(curve[:,0], points[0,0])
                end = np.searchsorted(curve[:,0], points[1,0])
                along2 = curve_length(curve[start:end])
                
                between.append(mean([between1,between2]))
                along.append(mean([along1,along2]))

                electrode.append([electrode1,electrode2])


                amp.append(lst.amp1[i])
                freq.append(lst.freq[i])

        df_investigate = pd.DataFrame({
                                       'double':double, 
                                       'freq':freq,
                                       'amp':amp,
                                        'distance':distance,
            'between':between,
            'along':along,
            
                                       'single_dtf':single_dtf,
                                       'single_dti':single_dti,
                                       'subject':lst_subject
                                      })
        
        df_AllSub = df_investigate.copy()
        
#         temp = pd.DataFrame()
#         for subj in df_investigate.subject.unique():
#             df_investigate_temp=df_investigate[df_investigate.subject == subj]
#             df_investigate_temp['double'] = (df_investigate_temp['double'] - df_investigate_temp['double'].mean()) / df_investigate_temp['double'].std()
#             df_investigate_temp['amp'] = (df_investigate_temp['amp'] - df_investigate_temp['amp'].mean()) / df_investigate_temp['amp'].std()
#             df_investigate_temp['single_dti'] = (df_investigate_temp['single_dti'] - df_investigate_temp['single_dti'].mean()) / df_investigate_temp['single_dti'].std()
#             df_investigate_temp['single_dtf'] = (df_investigate_temp['single_dtf'] - df_investigate_temp['single_dtf'].mean()) / df_investigate_temp['single_dtf'].std()
#             df_investigate_temp['between'] = (df_investigate_temp['between'] - df_investigate_temp['between'].mean()) / df_investigate_temp['between'].std()
#             df_investigate_temp['along'] = (df_investigate_temp['along'] - df_investigate_temp['along'].mean()) / df_investigate_temp['along'].std()
#             temp = pd.concat([temp, df_investigate_temp])
#         df_investigate = temp.copy().fillna(0).reset_index(drop=True)

# df_AllSub['double'] = (df_AllSub['double'] - df_AllSub['double'].mean()) / df_AllSub['double'].std()
# df_AllSub['amp'] = (df_AllSub['amp'] - df_AllSub['amp'].mean()) / df_AllSub['amp'].std()
# df_AllSub['single_dti'] = (df_AllSub['single_dti'] - df_AllSub['single_dti'].mean()) / df_AllSub['single_dti'].std()
# df_AllSub['single_dtf'] = (df_AllSub['single_dtf'] - df_AllSub['single_dtf'].mean()) / df_AllSub['single_dtf'].std()
# df_AllSub['between'] = (df_AllSub['between'] - df_AllSub['between'].mean()) / df_AllSub['between'].std()
# df_AllSub['along'] = (df_AllSub['along'] - df_AllSub['along'].mean()) / df_AllSub['along'].std()

        
            
        temp = df_investigate[(df_investigate.amp == 2) & (df_investigate.freq == 20) & 
                              (df_investigate.single_dti == 0) & 
                              (df_investigate.single_dtf > 2950) & (df_investigate.single_dtf < 3450)]
        temp = temp.groupby(['amp','freq','subject']).mean().reset_index()
        df = pd.DataFrame({})
        for subj in temp.subject:
            df_investigate_temp = df_investigate[df_investigate.subject == subj]
            df_investigate_temp['double'] = df_investigate_temp['double'] / temp[temp.subject == subj].iloc[0]['double']
            filename = subj + label_list[label]
            df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
            df = pd.concat([df, df_investigate_temp])
        df_investigate = df.copy()
        filename = label_list[label]
        df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')

In [ ]:
copy = df_investigate.copy()
df_AllSub=df_investigate.copy()
df_s = pd.DataFrame({})
for subj in ['12-005','51-009','52-001','All']:
    if subj=='All':
        df_t = df_AllSub.copy()
    else:   
        df_t = copy[copy.subject == subj]
    df_t['amp'] = (df_t['amp'] - df_t['amp'].mean()) / df_t['amp'].std()
    df_t['single_dti'] = (df_t['single_dti'] - df_t['single_dti'].mean()) / df_t['single_dti'].std()
    df_t['single_dtf'] = (df_t['single_dtf'] - df_t['single_dtf'].mean()) / df_t['single_dtf'].std()
    df_t['between'] = (df_t['between'] - df_t['between'].mean()) / df_t['between'].std()
    df_t['along'] = (df_t['along'] - df_t['along'].mean()) / df_t['along'].std()
    
    if subj == 'All':  
        df_AllSub = df_t.copy()
    else:
        df_s = pd.concat([df_s, df_t])
        df_s = df_s.fillna(0)
        
df_investigate = df_s.copy().reset_index(drop=True)

In [ ]:
fig, axs= plt.subplots(ncols=5, figsize=(35, 7))
column_lst = ['amp','single_dtf', 'single_dti','between','along']
name_lst = ['Amplitude','Foveal Eccentricity', 'Electrode-Implant Distance','Between-Axon Distance','Along-Axon Distance']
for dv in range(5):
    reg = LinearRegression().fit(df_investigate[column_lst[:dv] + column_lst[dv+1 :]], df_investigate[['double']])
    y_predicted = reg.predict(df_investigate[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_investigate[['double']]-y_predicted

    reg = LinearRegression().fit(df_investigate[column_lst[:dv] + column_lst[dv+1 :]], df_investigate[column_lst[dv]])
    y_predicted = reg.predict(df_investigate[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_investigate[column_lst[dv]]-y_predicted
    df_investigate['x'] = x
    df_investigate['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','s','^']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_investigate[df_investigate.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.6,markersize=13)
    axs[dv].legend(subject, loc='upper right',prop={'size': 15})
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
    axs[dv].set(xlabel = name_lst[dv], ylabel = 'Area',ylim=(-3,5))
    for item in ([axs[dv].xaxis.label, axs[dv].yaxis.label]):
        item.set_fontsize(20)
fig.savefig('/home/yuchen/paper/12a. Double-Electrode Area.pdf', transparent=True)

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == '12-005':
        X = temp[['amp','single_dtf','single_dti']]
    elif subj == 'AllSubjects':
        temp = df_AllSub.copy()
        X = temp[['amp','single_dtf','single_dti']]
    else:
        X = temp[['amp','single_dtf']]
    y = temp.double
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\nvariance inflation factor: ')
    print([[X.columns[i],variance_inflation_factor(X.values, i)]
                          for i in range(len(X.columns))])
    print('\npartial correlation: ')
    X['double'] = temp['double']
    print(X.pcorr())
    print(' ')

### major_axis_length

In [ ]:
df_AllSub = pd.DataFrame()

condition_list = [0]  
label_list = ['SpatialSummation','MultiElectrode']
label_list = ['MultiElectrode']
for condition in condition_list:
    for label in range(1):
        data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
        data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
        lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
        lst = lst.drop_duplicates().reset_index(drop=True)

        double = []
        single_dtf = []
        single_dti = []
        lst_subject = []
        
        distance = []
        amp = []
        freq = []
        electrode = []
        between = []
        along = []
        for i in range(len(lst)):
            electrode1 = lst.electrode1[i]
            electrode2 = lst.electrode2[i]
            temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
            if not temp1.empty and not temp2.empty and not double_temp.empty:
                count = 0
                for j in range(len(double_temp)):
                    count += sum(double_temp['major_axis_length'][j])
                double.append(count/len(double_temp))
                single_dtf.append((temp1.distance_to_fovea[0] + temp2.distance_to_fovea[0])/2)
                single_dti.append((temp1.distance_to_implant[0] + temp2.distance_to_implant[0])/2)
                lst_subject.append(temp1.subject[0])
                
                subject = shapes.subject_params[temp1.subject[0]]
                implant,model = shapes.model_from_params(subject)
                
                e1_x = implant[electrode1].x
                e1_y = implant[electrode1].y
                e2_x = implant[electrode2].x
                e2_y = implant[electrode2].y
                
                distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
                
                bundle = model.grow_axon_bundles()

                curve  = model.find_closest_axon(bundle, xret=e2_x, yret=e2_y)
                line = geom.LineString(curve)
                point1 = geom.Point(e1_x,e1_y)
                between1 = point1.distance(line)

                point_on_line1 = line.interpolate(line.project(point1))
                point2 = geom.Point(e2_x,e2_y)
                point_on_line2 = line.interpolate(line.project(point2))

                if point_on_line1.x < point_on_line2.x:
                    points = np.array([[point_on_line1.x,point_on_line1.y], [point_on_line2.x,point_on_line2.y]])
                else:
                    points = np.array([[point_on_line2.x,point_on_line2.y],[point_on_line1.x,point_on_line1.y]])
                curve = curve[::-1]
                start = np.searchsorted(curve[:,0], points[0,0])
                end = np.searchsorted(curve[:,0], points[1,0])
                along1 = curve_length(curve[start:end])
                
                curve  = model.find_closest_axon(bundle, xret=e1_x, yret=e1_y)
                line = geom.LineString(curve)
                point2 = geom.Point(e2_x,e2_y)
                between2 = point2.distance(line)
                
                point_on_line2 = line.interpolate(line.project(point2))
                point1 = geom.Point(e1_x,e1_y)
                point_on_line1 = line.interpolate(line.project(point1))

                if point_on_line2.x < point_on_line1.x:
                    points = np.array([[point_on_line2.x,point_on_line2.y], [point_on_line1.x,point_on_line1.y]])
                else:
                    points = np.array([[point_on_line1.x,point_on_line1.y],[point_on_line2.x,point_on_line2.y]])
                curve = curve[::-1]
                start = np.searchsorted(curve[:,0], points[0,0])
                end = np.searchsorted(curve[:,0], points[1,0])
                along2 = curve_length(curve[start:end])
                
                between.append(mean([between1,between2]))
                along.append(mean([along1,along2]))


                amp.append(lst.amp1[i])
                freq.append(lst.freq[i])

        df_investigate = pd.DataFrame({
                                       'double':double, 
                                       'freq':freq,
                                       'amp':amp,
                                        'distance':distance,
                                        'between':between,
                                        'along':along,
            
                                       'single_dtf':single_dtf,
                                       'single_dti':single_dti,
                                       'subject':lst_subject
                                      })
        # df_AllSub = df_investigate.copy()
        
#         temp = pd.DataFrame()
#         for subj in df_investigate.subject.unique():
#             df_investigate_temp=df_investigate[df_investigate.subject == subj]
#             df_investigate_temp['double'] = (df_investigate_temp['double'] - df_investigate_temp['double'].mean()) / df_investigate_temp['double'].std()
#             df_investigate_temp['amp'] = (df_investigate_temp['amp'] - df_investigate_temp['amp'].mean()) / df_investigate_temp['amp'].std()
#             df_investigate_temp['single_dti'] = (df_investigate_temp['single_dti'] - df_investigate_temp['single_dti'].mean()) / df_investigate_temp['single_dti'].std()
#             df_investigate_temp['single_dtf'] = (df_investigate_temp['single_dtf'] - df_investigate_temp['single_dtf'].mean()) / df_investigate_temp['single_dtf'].std()
#             df_investigate_temp['between'] = (df_investigate_temp['between'] - df_investigate_temp['between'].mean()) / df_investigate_temp['between'].std()
#             df_investigate_temp['along'] = (df_investigate_temp['along'] - df_investigate_temp['along'].mean()) / df_investigate_temp['along'].std()
#             temp = pd.concat([temp, df_investigate_temp])
#         df_investigate = temp.copy().fillna(0).reset_index(drop=True)

# df_AllSub['double'] = (df_AllSub['double'] - df_AllSub['double'].mean()) / df_AllSub['double'].std()
# df_AllSub['amp'] = (df_AllSub['amp'] - df_AllSub['amp'].mean()) / df_AllSub['amp'].std()
# df_AllSub['single_dti'] = (df_AllSub['single_dti'] - df_AllSub['single_dti'].mean()) / df_AllSub['single_dti'].std()
# df_AllSub['single_dtf'] = (df_AllSub['single_dtf'] - df_AllSub['single_dtf'].mean()) / df_AllSub['single_dtf'].std()
# df_AllSub['between'] = (df_AllSub['between'] - df_AllSub['between'].mean()) / df_AllSub['between'].std()
# df_AllSub['along'] = (df_AllSub['along'] - df_AllSub['along'].mean()) / df_AllSub['along'].std()

        
        temp = df_investigate[(df_investigate.amp == 2) & (df_investigate.freq == 20) & 
                              (df_investigate.single_dti == 0) & 
                              (df_investigate.single_dtf > 2950) & (df_investigate.single_dtf < 3450)]
        temp = temp.groupby(['amp','freq','subject']).mean().reset_index()
        df = pd.DataFrame({})
        for subj in temp.subject:
            df_investigate_temp = df_investigate[df_investigate.subject == subj]
            df_investigate_temp['double'] = df_investigate_temp['double'] / temp[temp.subject == subj].iloc[0]['double']
            filename = subj + label_list[label]
            df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
            df = pd.concat([df, df_investigate_temp])
        df_investigate = df.copy()
        filename = label_list[label]
        df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')

In [ ]:
copy = df_investigate.copy()
df_AllSub = df_investigate.copy()

df_s = pd.DataFrame({})
for subj in ['12-005','51-009','52-001','All']:
    if subj=='All':
        df_t = df_AllSub.copy()
    else:   
        df_t = copy[copy.subject == subj]
    df_t['amp'] = (df_t['amp'] - df_t['amp'].mean()) / df_t['amp'].std()
    df_t['single_dti'] = (df_t['single_dti'] - df_t['single_dti'].mean()) / df_t['single_dti'].std()
    df_t['single_dtf'] = (df_t['single_dtf'] - df_t['single_dtf'].mean()) / df_t['single_dtf'].std()
    df_t['between'] = (df_t['between'] - df_t['between'].mean()) / df_t['between'].std()
    df_t['along'] = (df_t['along'] - df_t['along'].mean()) / df_t['along'].std()
    
    if subj == 'All':  
        df_AllSub = df_t.copy()
    else:
        df_s = pd.concat([df_s, df_t])
        df_s = df_s.fillna(0)
        
df_investigate = df_s.copy().reset_index(drop=True)

In [ ]:
fig, axs= plt.subplots(ncols=5, figsize=(35, 7))
column_lst = ['amp','single_dtf', 'single_dti','between','along']
name_lst = ['Amplitude','Foveal Eccentricity', 'Electrode-Implant Distance','Between-Axon Distance','Along-Axon Distance']
for dv in range(5):
    reg = LinearRegression().fit(df_investigate[column_lst[:dv] + column_lst[dv+1 :]], df_investigate[['double']])
    y_predicted = reg.predict(df_investigate[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_investigate[['double']]-y_predicted

    reg = LinearRegression().fit(df_investigate[column_lst[:dv] + column_lst[dv+1 :]], df_investigate[column_lst[dv]])
    y_predicted = reg.predict(df_investigate[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_investigate[column_lst[dv]]-y_predicted
    df_investigate['x'] = x
    df_investigate['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','s','^']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_investigate[df_investigate.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.6,markersize=13)
    axs[dv].legend(subject, loc='upper right',prop={'size': 15})
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
    axs[dv].set(xlabel = name_lst[dv], ylabel = 'Major Axis Length',ylim=(-0.7,1.1))
    for item in ([axs[dv].xaxis.label, axs[dv].yaxis.label]):
        item.set_fontsize(20)
fig.savefig('/home/yuchen/paper/12b. Double-Electrode Major Axis Length.pdf', transparent=True)

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == '12-005':
        X = temp[['amp','single_dtf','single_dti']]
    elif subj == 'AllSubjects':
        temp = df_AllSub.copy()
        X = temp[['amp','single_dtf','single_dti']]
    else:
        X = temp[['amp','single_dtf']]
    y = temp.double
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\nvariance inflation factor: ')
    print([[X.columns[i],variance_inflation_factor(X.values, i)]
                          for i in range(len(X.columns))])
    print('\npartial correlation: ')
    X['double'] = temp['double']
    print(X.pcorr())
    print(' ')

### minor_axis_length

In [ ]:
df_AllSub = pd.DataFrame()

condition_list = [0]  
label_list = ['MultiElectrode']
for condition in condition_list:
    for label in range(1):
        data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
        data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
        lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
        lst = lst.drop_duplicates().reset_index(drop=True)

        double = []
        single_dtf = []
        single_dti = []
        lst_subject = []
        
        distance = []
        amp = []
        freq = []
        electrode = []
        between = []
        along = []
        for i in range(len(lst)):
            electrode1 = lst.electrode1[i]
            electrode2 = lst.electrode2[i]
            temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
            if not temp1.empty and not temp2.empty and not double_temp.empty:
                count = 0
                for j in range(len(double_temp)):
                    count += sum(double_temp['minor_axis_length'][j])
                double.append(count/len(double_temp))
                single_dtf.append((temp1.distance_to_fovea[0] + temp2.distance_to_fovea[0])/2)
                single_dti.append((temp1.distance_to_implant[0] + temp2.distance_to_implant[0])/2)
                lst_subject.append(temp1.subject[0])
                
                subject = shapes.subject_params[temp1.subject[0]]
                implant,model = shapes.model_from_params(subject)
                
                e1_x = implant[electrode1].x
                e1_y = implant[electrode1].y
                e2_x = implant[electrode2].x
                e2_y = implant[electrode2].y
                
                distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
                
                bundle = model.grow_axon_bundles()

                curve  = model.find_closest_axon(bundle, xret=e2_x, yret=e2_y)
                line = geom.LineString(curve)
                point1 = geom.Point(e1_x,e1_y)
                between1 = point1.distance(line)

                point_on_line1 = line.interpolate(line.project(point1))
                point2 = geom.Point(e2_x,e2_y)
                point_on_line2 = line.interpolate(line.project(point2))

                if point_on_line1.x < point_on_line2.x:
                    points = np.array([[point_on_line1.x,point_on_line1.y], [point_on_line2.x,point_on_line2.y]])
                else:
                    points = np.array([[point_on_line2.x,point_on_line2.y],[point_on_line1.x,point_on_line1.y]])
                curve = curve[::-1]
                start = np.searchsorted(curve[:,0], points[0,0])
                end = np.searchsorted(curve[:,0], points[1,0])
                along1 = curve_length(curve[start:end])
                
                curve  = model.find_closest_axon(bundle, xret=e1_x, yret=e1_y)
                line = geom.LineString(curve)
                point2 = geom.Point(e2_x,e2_y)
                between2 = point2.distance(line)
                
                point_on_line2 = line.interpolate(line.project(point2))
                point1 = geom.Point(e1_x,e1_y)
                point_on_line1 = line.interpolate(line.project(point1))

                if point_on_line2.x < point_on_line1.x:
                    points = np.array([[point_on_line2.x,point_on_line2.y], [point_on_line1.x,point_on_line1.y]])
                else:
                    points = np.array([[point_on_line1.x,point_on_line1.y],[point_on_line2.x,point_on_line2.y]])
                curve = curve[::-1]
                start = np.searchsorted(curve[:,0], points[0,0])
                end = np.searchsorted(curve[:,0], points[1,0])
                along2 = curve_length(curve[start:end])
                
                between.append(mean([between1,between2]))
                along.append(mean([along1,along2]))

                amp.append(lst.amp1[i])
                freq.append(lst.freq[i])

        df_investigate = pd.DataFrame({
                                       'double':double, 
                                       'freq':freq,
                                       'amp':amp,
                                        'distance':distance,
                                        'between':between,
                                        'along':along,
            
                                       'single_dtf':single_dtf,
                                       'single_dti':single_dti,
                                       'subject':lst_subject
                                      })
#         df_AllSub = df_investigate.copy()
        
#         temp = pd.DataFrame()
#         for subj in df_investigate.subject.unique():
#             df_investigate_temp=df_investigate[df_investigate.subject == subj]
#             df_investigate_temp['double'] = (df_investigate_temp['double'] - df_investigate_temp['double'].mean()) / df_investigate_temp['double'].std()
#             df_investigate_temp['amp'] = (df_investigate_temp['amp'] - df_investigate_temp['amp'].mean()) / df_investigate_temp['amp'].std()
#             df_investigate_temp['single_dti'] = (df_investigate_temp['single_dti'] - df_investigate_temp['single_dti'].mean()) / df_investigate_temp['single_dti'].std()
#             df_investigate_temp['single_dtf'] = (df_investigate_temp['single_dtf'] - df_investigate_temp['single_dtf'].mean()) / df_investigate_temp['single_dtf'].std()
#             df_investigate_temp['between'] = (df_investigate_temp['between'] - df_investigate_temp['between'].mean()) / df_investigate_temp['between'].std()
#             df_investigate_temp['along'] = (df_investigate_temp['along'] - df_investigate_temp['along'].mean()) / df_investigate_temp['along'].std()
#             temp = pd.concat([temp, df_investigate_temp])
#         df_investigate = temp.copy().fillna(0).reset_index(drop=True)

# df_AllSub['double'] = (df_AllSub['double'] - df_AllSub['double'].mean()) / df_AllSub['double'].std()
# df_AllSub['amp'] = (df_AllSub['amp'] - df_AllSub['amp'].mean()) / df_AllSub['amp'].std()
# df_AllSub['single_dti'] = (df_AllSub['single_dti'] - df_AllSub['single_dti'].mean()) / df_AllSub['single_dti'].std()
# df_AllSub['single_dtf'] = (df_AllSub['single_dtf'] - df_AllSub['single_dtf'].mean()) / df_AllSub['single_dtf'].std()
# df_AllSub['between'] = (df_AllSub['between'] - df_AllSub['between'].mean()) / df_AllSub['between'].std()
# df_AllSub['along'] = (df_AllSub['along'] - df_AllSub['along'].mean()) / df_AllSub['along'].std()

        
        temp = df_investigate[(df_investigate.amp == 2) & (df_investigate.freq == 20) & 
                              (df_investigate.single_dti == 0) & 
                              (df_investigate.single_dtf > 2950) & (df_investigate.single_dtf < 3450)]
        temp = temp.groupby(['amp','freq','subject']).mean().reset_index()
        df = pd.DataFrame({})
        for subj in temp.subject:
            df_investigate_temp = df_investigate[df_investigate.subject == subj]
            df_investigate_temp['double'] = df_investigate_temp['double'] / temp[temp.subject == subj].iloc[0]['double']
            filename = subj + label_list[label]
            df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
            df = pd.concat([df, df_investigate_temp])
        df_investigate = df.copy()
        filename = label_list[label]
        df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')

In [ ]:
copy = df_investigate.copy()
df_AllSub = df_investigate.copy()

df_s = pd.DataFrame({})
for subj in ['12-005','51-009','52-001','All']:
    if subj=='All':
        df_t = df_AllSub.copy()
    else:   
        df_t = copy[copy.subject == subj]
    df_t['amp'] = (df_t['amp'] - df_t['amp'].mean()) / df_t['amp'].std()
    df_t['single_dti'] = (df_t['single_dti'] - df_t['single_dti'].mean()) / df_t['single_dti'].std()
    df_t['single_dtf'] = (df_t['single_dtf'] - df_t['single_dtf'].mean()) / df_t['single_dtf'].std()
    df_t['between'] = (df_t['between'] - df_t['between'].mean()) / df_t['between'].std()
    df_t['along'] = (df_t['along'] - df_t['along'].mean()) / df_t['along'].std()
    
    if subj == 'All':  
        df_AllSub = df_t.copy()
    else:
        df_s = pd.concat([df_s, df_t])
        df_s = df_s.fillna(0)
        
df_investigate = df_s.copy().reset_index(drop=True)

In [ ]:
fig, axs= plt.subplots(ncols=5, figsize=(35, 7))
column_lst = ['amp','single_dtf', 'single_dti','between','along']
name_lst = ['Amplitude','Foveal Eccentricity', 'Electrode-Implant Distance','Between-Axon Distance','Along-Axon Distance']
for dv in range(5):
    reg = LinearRegression().fit(df_investigate[column_lst[:dv] + column_lst[dv+1 :]], df_investigate[['double']])
    y_predicted = reg.predict(df_investigate[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_investigate[['double']]-y_predicted

    reg = LinearRegression().fit(df_investigate[column_lst[:dv] + column_lst[dv+1 :]], df_investigate[column_lst[dv]])
    y_predicted = reg.predict(df_investigate[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_investigate[column_lst[dv]]-y_predicted
    df_investigate['x'] = x
    df_investigate['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','s','^']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_investigate[df_investigate.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.6,markersize=13)
    axs[dv].legend(subject, loc='upper right',prop={'size': 15})
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
    axs[dv].set(xlabel = name_lst[dv], ylabel = 'Minor Axis Length',ylim=(-1.5,2.1))
    for item in ([axs[dv].xaxis.label, axs[dv].yaxis.label]):
        item.set_fontsize(20)
fig.savefig('/home/yuchen/paper/12c. Double-Electrode Minor Axis Length.pdf', transparent=True)

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == '12-005':
        X = temp[['amp','single_dtf','single_dti']]
    elif subj == 'AllSubjects':
        temp = df_AllSub.copy()
        X = temp[['amp','single_dtf','single_dti']]
    else:
        X = temp[['amp','single_dtf']]
    y = temp.double
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\nvariance inflation factor: ')
    print([[X.columns[i],variance_inflation_factor(X.values, i)]
                          for i in range(len(X.columns))])
    print('\npartial correlation: ')
    X['double'] = temp['double']
    print(X.pcorr())
    print(' ')

### perimeter

In [ ]:
df_AllSub = pd.DataFrame()

condition_list = [0]  
label_list = ['MultiElectrode']
for condition in condition_list:
    for label in range(1):
        data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
        data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
        lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
        lst = lst.drop_duplicates().reset_index(drop=True)

        double = []
        single_dtf = []
        single_dti = []
        lst_subject = []
        
        distance = []
        amp = []
        freq = []
        electrode = []
        between = []
        along = []
        for i in range(len(lst)):
            electrode1 = lst.electrode1[i]
            electrode2 = lst.electrode2[i]
            temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
            if not temp1.empty and not temp2.empty and not double_temp.empty:
                count = 0
                for j in range(len(double_temp)):
                    count += sum(double_temp['perimeter'][j])
                double.append(count/len(double_temp))
                single_dtf.append((temp1.distance_to_fovea[0] + temp2.distance_to_fovea[0])/2)
                single_dti.append((temp1.distance_to_implant[0] + temp2.distance_to_implant[0])/2)
                lst_subject.append(temp1.subject[0])
                
                subject = shapes.subject_params[temp1.subject[0]]
                implant,model = shapes.model_from_params(subject)
                
                e1_x = implant[electrode1].x
                e1_y = implant[electrode1].y
                e2_x = implant[electrode2].x
                e2_y = implant[electrode2].y
                
                distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
                
                bundle = model.grow_axon_bundles()

                curve  = model.find_closest_axon(bundle, xret=e2_x, yret=e2_y)
                line = geom.LineString(curve)
                point1 = geom.Point(e1_x,e1_y)
                between1 = point1.distance(line)

                point_on_line1 = line.interpolate(line.project(point1))
                point2 = geom.Point(e2_x,e2_y)
                point_on_line2 = line.interpolate(line.project(point2))

                if point_on_line1.x < point_on_line2.x:
                    points = np.array([[point_on_line1.x,point_on_line1.y], [point_on_line2.x,point_on_line2.y]])
                else:
                    points = np.array([[point_on_line2.x,point_on_line2.y],[point_on_line1.x,point_on_line1.y]])
                curve = curve[::-1]
                start = np.searchsorted(curve[:,0], points[0,0])
                end = np.searchsorted(curve[:,0], points[1,0])
                along1 = curve_length(curve[start:end])
                
                curve  = model.find_closest_axon(bundle, xret=e1_x, yret=e1_y)
                line = geom.LineString(curve)
                point2 = geom.Point(e2_x,e2_y)
                between2 = point2.distance(line)
                
                point_on_line2 = line.interpolate(line.project(point2))
                point1 = geom.Point(e1_x,e1_y)
                point_on_line1 = line.interpolate(line.project(point1))

                if point_on_line2.x < point_on_line1.x:
                    points = np.array([[point_on_line2.x,point_on_line2.y], [point_on_line1.x,point_on_line1.y]])
                else:
                    points = np.array([[point_on_line1.x,point_on_line1.y],[point_on_line2.x,point_on_line2.y]])
                curve = curve[::-1]
                start = np.searchsorted(curve[:,0], points[0,0])
                end = np.searchsorted(curve[:,0], points[1,0])
                along2 = curve_length(curve[start:end])
                
                between.append(mean([between1,between2]))
                along.append(mean([along1,along2]))
                
                amp.append(lst.amp1[i])
                freq.append(lst.freq[i])

        df_investigate = pd.DataFrame({
                                       'double':double, 
                                       'freq':freq,
                                       'amp':amp,
                                        'distance':distance,
                                        'between':between,
                                        'along':along,
            
                                       'single_dtf':single_dtf,
                                       'single_dti':single_dti,
                                       'subject':lst_subject
                                      })
#         df_AllSub = df_investigate.copy()
        
#         temp = pd.DataFrame()
#         for subj in df_investigate.subject.unique():
#             df_investigate_temp=df_investigate[df_investigate.subject == subj]
#             df_investigate_temp['double'] = (df_investigate_temp['double'] - df_investigate_temp['double'].mean()) / df_investigate_temp['double'].std()
#             df_investigate_temp['amp'] = (df_investigate_temp['amp'] - df_investigate_temp['amp'].mean()) / df_investigate_temp['amp'].std()
#             df_investigate_temp['single_dti'] = (df_investigate_temp['single_dti'] - df_investigate_temp['single_dti'].mean()) / df_investigate_temp['single_dti'].std()
#             df_investigate_temp['single_dtf'] = (df_investigate_temp['single_dtf'] - df_investigate_temp['single_dtf'].mean()) / df_investigate_temp['single_dtf'].std()
#             df_investigate_temp['between'] = (df_investigate_temp['between'] - df_investigate_temp['between'].mean()) / df_investigate_temp['between'].std()
#             df_investigate_temp['along'] = (df_investigate_temp['along'] - df_investigate_temp['along'].mean()) / df_investigate_temp['along'].std()
#             temp = pd.concat([temp, df_investigate_temp])
#         df_investigate = temp.copy().fillna(0).reset_index(drop=True)

# df_AllSub['double'] = (df_AllSub['double'] - df_AllSub['double'].mean()) / df_AllSub['double'].std()
# df_AllSub['amp'] = (df_AllSub['amp'] - df_AllSub['amp'].mean()) / df_AllSub['amp'].std()
# df_AllSub['single_dti'] = (df_AllSub['single_dti'] - df_AllSub['single_dti'].mean()) / df_AllSub['single_dti'].std()
# df_AllSub['single_dtf'] = (df_AllSub['single_dtf'] - df_AllSub['single_dtf'].mean()) / df_AllSub['single_dtf'].std()
# df_AllSub['between'] = (df_AllSub['between'] - df_AllSub['between'].mean()) / df_AllSub['between'].std()
# df_AllSub['along'] = (df_AllSub['along'] - df_AllSub['along'].mean()) / df_AllSub['along'].std()

        
        temp = df_investigate[(df_investigate.amp == 2) & (df_investigate.freq == 20) & 
                              (df_investigate.single_dti == 0) & 
                              (df_investigate.single_dtf > 2950) & (df_investigate.single_dtf < 3450)]
        temp = temp.groupby(['amp','freq','subject']).mean().reset_index()
        df = pd.DataFrame({})
        for subj in temp.subject:
            df_investigate_temp = df_investigate[df_investigate.subject == subj]
            df_investigate_temp['double'] = df_investigate_temp['double'] / temp[temp.subject == subj].iloc[0]['double']
            filename = subj + label_list[label]
            df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
            df = pd.concat([df, df_investigate_temp])
        df_investigate = df.copy()
        filename = label_list[label]
        df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')

In [ ]:
copy = df_investigate.copy()
df_AllSub = df_investigate.copy()

df_s = pd.DataFrame({})
for subj in ['12-005','51-009','52-001','All']:
    if subj=='All':
        df_t = df_AllSub.copy()
    else:   
        df_t = copy[copy.subject == subj]
    df_t['amp'] = (df_t['amp'] - df_t['amp'].mean()) / df_t['amp'].std()
    df_t['single_dti'] = (df_t['single_dti'] - df_t['single_dti'].mean()) / df_t['single_dti'].std()
    df_t['single_dtf'] = (df_t['single_dtf'] - df_t['single_dtf'].mean()) / df_t['single_dtf'].std()
    df_t['between'] = (df_t['between'] - df_t['between'].mean()) / df_t['between'].std()
    df_t['along'] = (df_t['along'] - df_t['along'].mean()) / df_t['along'].std()
    
    if subj == 'All':  
        df_AllSub = df_t.copy()
    else:
        df_s = pd.concat([df_s, df_t])
        df_s = df_s.fillna(0)
        
df_investigate = df_s.copy().reset_index(drop=True)

In [ ]:
fig, axs= plt.subplots(ncols=5, figsize=(35, 7))
column_lst = ['amp','single_dtf', 'single_dti','between','along']
name_lst = ['Amplitude','Foveal Eccentricity', 'Electrode-Implant Distance','Between-Axon Distance','Along-Axon Distance']
for dv in range(5):
    reg = LinearRegression().fit(df_investigate[column_lst[:dv] + column_lst[dv+1 :]], df_investigate[['double']])
    y_predicted = reg.predict(df_investigate[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_investigate[['double']]-y_predicted

    reg = LinearRegression().fit(df_investigate[column_lst[:dv] + column_lst[dv+1 :]], df_investigate[column_lst[dv]])
    y_predicted = reg.predict(df_investigate[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_investigate[column_lst[dv]]-y_predicted
    df_investigate['x'] = x
    df_investigate['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','s','^']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_investigate[df_investigate.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.6,markersize=13)
    axs[dv].legend(subject, loc='upper right',prop={'size': 15})
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
    axs[dv].set(xlabel = name_lst[dv], ylabel = 'Perimeter',ylim=(-0.7,1.1))
    for item in ([axs[dv].xaxis.label, axs[dv].yaxis.label]):
        item.set_fontsize(20)
fig.savefig('/home/yuchen/paper/12d. Double-Electrode Perimeter.pdf', transparent=True)

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == '12-005':
        X = temp[['amp','single_dtf','single_dti']]
    elif subj == 'AllSubjects':
        temp = df_AllSub.copy()
        X = temp[['amp','single_dtf','single_dti']]
    else:
        X = temp[['amp','single_dtf']]
    y = temp.double
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\nvariance inflation factor: ')
    print([[X.columns[i],variance_inflation_factor(X.values, i)]
                          for i in range(len(X.columns))])
    print('\npartial correlation: ')
    X['double'] = temp['double']
    print(X.pcorr())
    print(' ')

In [ ]:
est2.pvalues

### orientation

In [ ]:
df_AllSub = pd.DataFrame()

condition_list = [0]  
label_list = ['MultiElectrode']
for condition in condition_list:
    for label in range(1):
        data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
        data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
        lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
        lst = lst.drop_duplicates().reset_index(drop=True)

        double = []
        single_dtf = []
        single_dti = []
        lst_subject = []
        between = []
        along = []
        
        distance = []
        amp = []
        freq = []
        tan = []
        electrode = []
        for i in range(len(lst)):
            electrode1 = lst.electrode1[i]
            electrode2 = lst.electrode2[i]
            temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
            if not temp1.empty and not temp2.empty and not double_temp.empty:
                count = 0
                for j in range(len(double_temp)):
                    count += mean(double_temp['orientation_new'][j])
                double.append(count/len(double_temp))
                single_dtf.append((temp1.distance_to_fovea[0] + temp2.distance_to_fovea[0])/2)
                single_dti.append((temp1.distance_to_implant[0] + temp2.distance_to_implant[0])/2)
                lst_subject.append(temp1.subject[0])
                
                subject = shapes.subject_params[temp1.subject[0]]
                implant,model = shapes.model_from_params(subject)
                
                e1_x = implant[electrode1].x
                e1_y = implant[electrode1].y
                e2_x = implant[electrode2].x
                e2_y = implant[electrode2].y
                
                tan.append(mean([model.calc_bundle_tangent(e1_x, e1_y)*-1, model.calc_bundle_tangent(e2_x, e2_y)*-1]))
                
                distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
                
                bundle = model.grow_axon_bundles()

                curve  = model.find_closest_axon(bundle, xret=e2_x, yret=e2_y)
                line = geom.LineString(curve)
                point1 = geom.Point(e1_x,e1_y)
                between1 = point1.distance(line)

                point_on_line1 = line.interpolate(line.project(point1))
                point2 = geom.Point(e2_x,e2_y)
                point_on_line2 = line.interpolate(line.project(point2))

                if point_on_line1.x < point_on_line2.x:
                    points = np.array([[point_on_line1.x,point_on_line1.y], [point_on_line2.x,point_on_line2.y]])
                else:
                    points = np.array([[point_on_line2.x,point_on_line2.y],[point_on_line1.x,point_on_line1.y]])
                curve = curve[::-1]
                start = np.searchsorted(curve[:,0], points[0,0])
                end = np.searchsorted(curve[:,0], points[1,0])
                along1 = curve_length(curve[start:end])
                
                curve  = model.find_closest_axon(bundle, xret=e1_x, yret=e1_y)
                line = geom.LineString(curve)
                point2 = geom.Point(e2_x,e2_y)
                between2 = point2.distance(line)
                
                point_on_line2 = line.interpolate(line.project(point2))
                point1 = geom.Point(e1_x,e1_y)
                point_on_line1 = line.interpolate(line.project(point1))

                if point_on_line2.x < point_on_line1.x:
                    points = np.array([[point_on_line2.x,point_on_line2.y], [point_on_line1.x,point_on_line1.y]])
                else:
                    points = np.array([[point_on_line1.x,point_on_line1.y],[point_on_line2.x,point_on_line2.y]])
                curve = curve[::-1]
                start = np.searchsorted(curve[:,0], points[0,0])
                end = np.searchsorted(curve[:,0], points[1,0])
                along2 = curve_length(curve[start:end])
                
                between.append(mean([between1,between2]))
                along.append(mean([along1,along2]))

                amp.append(lst.amp1[i])
                freq.append(lst.freq[i])

        df_investigate = pd.DataFrame({
                                       'double':double, 
                                       'freq':freq,
                                       'amp':amp,
                                        'distance':distance,
                                        'between':between,
                                        'along':along,
            'tan':tan,
            
                                       'single_dtf':single_dtf,
                                       'single_dti':single_dti,
                                       'subject':lst_subject
                                      })
        for subj in ['12-005', '51-009', '52-001']:
            df_investigate_temp = df_investigate[df_investigate.subject == subj]
            filename = subj + label_list[label]
            df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
        filename = label_list[label]
        df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
        
df_AllSub = df_investigate.copy()

In [ ]:
for i in range(len(df_investigate)):
    if df_investigate.tan[i] < 0:
        df_investigate.at[i,'tan'] = df_investigate.tan[i]+np.pi
    if df_investigate.double[i] < 0:
        df_investigate.at[i,'double'] = df_investigate.double[i]+np.pi
# for subj in ['12-005', '51-009', '52-001']:
#     df_investigate_temp = df_investigate[df_investigate.subject == subj]
#     filename = subj + label_list[label]
#     df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
# filename = label_list[label]
# df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')

In [ ]:
copy = df_investigate.copy()
df_AllSub = df_investigate.copy()

df_s = pd.DataFrame({})
for subj in ['12-005','51-009','52-001','All']:
    if subj=='All':
        df_t = df_AllSub.copy()
    else:   
        df_t = copy[copy.subject == subj]
    df_t['amp'] = (df_t['amp'] - df_t['amp'].mean()) / df_t['amp'].std()
    df_t['single_dti'] = (df_t['single_dti'] - df_t['single_dti'].mean()) / df_t['single_dti'].std()
    df_t['single_dtf'] = (df_t['single_dtf'] - df_t['single_dtf'].mean()) / df_t['single_dtf'].std()
    df_t['between'] = (df_t['between'] - df_t['between'].mean()) / df_t['between'].std()
    df_t['along'] = (df_t['along'] - df_t['along'].mean()) / df_t['along'].std()
    
    if subj == 'All':  
        df_AllSub = df_t.copy()
    else:
        df_s = pd.concat([df_s, df_t])
        df_s = df_s.fillna(0)
        
df_investigate = df_s.copy().reset_index(drop=True)

In [ ]:
fig, axs= plt.subplots(ncols=6, figsize=(42, 7))
column_lst = ['amp','single_dtf', 'single_dti','between','along','tan']
name_lst = ['Amplitude','Foveal Eccentricity', 'Electrode-Implant Distance','Between-Axon Distance','Along-Axon Distance', 'Axonal Tangent Line']
for dv in range(6):
    reg = LinearRegression().fit(df_investigate[column_lst[:dv] + column_lst[dv+1 :]], df_investigate[['double']])
    y_predicted = reg.predict(df_investigate[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_investigate[['double']]-y_predicted

    reg = LinearRegression().fit(df_investigate[column_lst[:dv] + column_lst[dv+1 :]], df_investigate[column_lst[dv]])
    y_predicted = reg.predict(df_investigate[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_investigate[column_lst[dv]]-y_predicted
    df_investigate['x'] = x
    df_investigate['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','s','^']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_investigate[df_investigate.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.6,markersize=13)
    axs[dv].legend(subject, loc='upper right',prop={'size': 15})
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
    axs[dv].set(xlabel = name_lst[dv], ylabel = 'Orientation',ylim=(-2.3,2.3))
    for item in ([axs[dv].xaxis.label, axs[dv].yaxis.label]):
        item.set_fontsize(20)
fig.savefig('/home/yuchen/paper/12e. Double-Electrode Orientation.pdf', transparent=True)

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == '12-005':
        X = temp[['amp','single_dtf','single_dti','tan','between','along']]
    elif subj == 'AllSubjects':
        temp = df_AllSub.copy()
        X = temp[['amp','single_dtf','single_dti','tan','between','along']]
    else:
        X = temp[['amp','single_dtf','tan','between','along']]
    y = temp.double
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\nvariance inflation factor: ')
    print([[X.columns[i],variance_inflation_factor(X.values, i)]
                          for i in range(len(X.columns))])
    print('\npartial correlation: ')
    print(temp[['amp','single_dtf','single_dti','tan','between','along','double']].pcorr())
    print(' ')

### eccentricity

In [ ]:
condition_list = [0]  
label_list = ['SpatialSummation','MultiElectrode']
label_list = ['MultiElectrode']
for condition in condition_list:
    for label in range(1):
        data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
        data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
        lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
        lst = lst.drop_duplicates().reset_index(drop=True)

        double = []
        single_dtf = []
        single_dti = []
        lst_subject = []
        
        distance = []
        between = []
        along = []
        amp = []
        freq = []
        electrode = []
        for i in range(len(lst)):
            electrode1 = lst.electrode1[i]
            electrode2 = lst.electrode2[i]
            temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
            double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
            if not temp1.empty and not temp2.empty and not double_temp.empty:
                count = 0
                for j in range(len(double_temp)):
                    count += mean(double_temp['eccentricity_new'][j])
                double.append(count/len(double_temp))
                single_dtf.append((temp1.distance_to_fovea[0] + temp2.distance_to_fovea[0])/2)
                single_dti.append((temp1.distance_to_implant[0] + temp2.distance_to_implant[0])/2)
                lst_subject.append(temp1.subject[0])
                
                subject = shapes.subject_params[temp1.subject[0]]
                implant,model = shapes.model_from_params(subject)
                
                e1_x = implant[electrode1].x
                e1_y = implant[electrode1].y
                e2_x = implant[electrode2].x
                e2_y = implant[electrode2].y
                
                distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
                
                bundle = model.grow_axon_bundles()

                curve  = model.find_closest_axon(bundle, xret=e2_x, yret=e2_y)
                line = geom.LineString(curve)
                point1 = geom.Point(e1_x,e1_y)
                between1 = point1.distance(line)

                point_on_line1 = line.interpolate(line.project(point1))
                point2 = geom.Point(e2_x,e2_y)
                point_on_line2 = line.interpolate(line.project(point2))

                if point_on_line1.x < point_on_line2.x:
                    points = np.array([[point_on_line1.x,point_on_line1.y], [point_on_line2.x,point_on_line2.y]])
                else:
                    points = np.array([[point_on_line2.x,point_on_line2.y],[point_on_line1.x,point_on_line1.y]])
                curve = curve[::-1]
                start = np.searchsorted(curve[:,0], points[0,0])
                end = np.searchsorted(curve[:,0], points[1,0])
                along1 = curve_length(curve[start:end])
                
                curve  = model.find_closest_axon(bundle, xret=e1_x, yret=e1_y)
                line = geom.LineString(curve)
                point2 = geom.Point(e2_x,e2_y)
                between2 = point2.distance(line)
                
                point_on_line2 = line.interpolate(line.project(point2))
                point1 = geom.Point(e1_x,e1_y)
                point_on_line1 = line.interpolate(line.project(point1))

                if point_on_line2.x < point_on_line1.x:
                    points = np.array([[point_on_line2.x,point_on_line2.y], [point_on_line1.x,point_on_line1.y]])
                else:
                    points = np.array([[point_on_line1.x,point_on_line1.y],[point_on_line2.x,point_on_line2.y]])
                curve = curve[::-1]
                start = np.searchsorted(curve[:,0], points[0,0])
                end = np.searchsorted(curve[:,0], points[1,0])
                along2 = curve_length(curve[start:end])
                
                between.append(mean([between1, between2]))
                along.append(mean([along1, along2]))

                amp.append(lst.amp1[i])
                freq.append(lst.freq[i])

        df_investigate = pd.DataFrame({
                                       'double':double, 
                                       'freq':freq,
                                       'amp':amp,
                                        'distance':distance,
                                        'between':between,
                                        'along':along,
            
                                       'single_dtf':single_dtf,
                                       'single_dti':single_dti,
                                       'subject':lst_subject
                                      })
        for subj in ['12-005', '51-009', '52-001']:
            df_investigate_temp = df_investigate[df_investigate.subject == subj]
            filename = subj + label_list[label]
            df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
        filename =  label_list[label]
        df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')

In [ ]:
fig, axs= plt.subplots(ncols=6, figsize=(42, 7))
column_lst = ['amp', 'freq','single_dtf', 'single_dti','between','along']
name_lst = ['Amplitude','Frequency','Foveal Eccentricity', 'Electrode-Implant Distance','Between-Axon Distance','Along-Axon Distance']
for dv in range(6):
    reg = LinearRegression().fit(df_investigate[column_lst[:dv] + column_lst[dv+1 :]], df_investigate[['double']])
    y_predicted = reg.predict(df_investigate[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_investigate[['double']]-y_predicted

    reg = LinearRegression().fit(df_investigate[column_lst[:dv] + column_lst[dv+1 :]], df_investigate[column_lst[dv]])
    y_predicted = reg.predict(df_investigate[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_investigate[column_lst[dv]]-y_predicted
    df_investigate['x'] = x
    df_investigate['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','X','s']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_investigate[df_investigate.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.8)
    axs[dv].legend(subject, loc='upper right')
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
    axs[dv].text(x=x.min(),y=y.max(), s='r = '+ str(np.corrcoef(df_investigate.x,df_investigate.y)[0,1].round(3)))
    axs[dv].set(xlabel = name_lst[dv], ylabel = 'Orientation (Double Electrode)')

In [ ]:
fig, axs= plt.subplots(ncols=4, figsize=(28, 7))
column_lst = ['amp', 'freq','single_dtf', 'single_dti']
name_lst = ['Amplitude','Frequency','Foveal Eccentricity', 'Electrode-Implant Distance']
for dv in range(4):
    reg = LinearRegression().fit(df_investigate[column_lst[:dv] + column_lst[dv+1 :]], df_investigate[['double']])
    y_predicted = reg.predict(df_investigate[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_investigate[['double']]-y_predicted

    reg = LinearRegression().fit(df_investigate[column_lst[:dv] + column_lst[dv+1 :]], df_investigate[column_lst[dv]])
    y_predicted = reg.predict(df_investigate[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_investigate[column_lst[dv]]-y_predicted
    df_investigate['x'] = x
    df_investigate['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','X','s']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_investigate[df_investigate.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.8)
    axs[dv].legend(subject, loc='upper right')
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
    axs[dv].text(x=x.min(),y=y.max(), s='r = '+ str(np.corrcoef(df_investigate.x,df_investigate.y)[0,1].round(3)))
    axs[dv].set(xlabel = name_lst[dv], ylabel = 'Eccentricity (Double Electrode)')

### number of phosphene

In [ ]:
data = data_copy.copy()

data_double = data[(data['electrode2'] != str())].reset_index(drop=True)
data_single = data[(data['electrode2'] == str())].reset_index(drop=True)
data_single['stim_class'] ='SingleElectrode'
df = data_single[['subject', 'electrode1', 'freq', 'amp1']].drop_duplicates().reset_index(drop=True)
lst = []
lst_phosphene = []


for i in range(len(df)):
    df_temp = data_single[(data_single['subject'] == df.subject[i]) & (data_single['electrode1'] == df.electrode1[i]) & (data_single['freq'] == df.freq[i]) & (data_single['amp1'] == df.amp1[i])].reset_index(drop=True)
    ratio = len(df_temp[df_temp['num_regions']>1]) /len(df_temp)
    lst.append(ratio)
    lst_phosphene.append(mean(df_temp['num_regions']))
    


df['helper'] = lst
df['phosphene'] = lst_phosphene
data_single = df

subject = ['12-005', '51-009', '52-001']
df = pd.DataFrame({})

for subj in subject:
    lst_e = []
    lst_dtf = []
    
    s2 = shapes.subject_params[subj]
    implant,model = shapes.model_from_params(s2)
    for i in string.ascii_uppercase[0:6]: 
        for j in range(1,11):
            electrode = i + str(j)
            lst_e.append(electrode)
            lst_dtf.append(math.sqrt(implant[electrode].x**2 +implant[electrode].y**2 ))
    df_o = pd.DataFrame(lst_e, columns=['electrode1'])
    df_o['distance_to_fovea'] = lst_dtf
    
    if subj == '12-005':
        lst_d = [0,0,0,2,7,7,0,0,5,0,
                0,0,4,11,13,15,15,3,7,4,
                0,0,15,16,17,17,19,16,9,4,
                0,0,16,19,15,17,22,25,13,10,
                0,0,8,15,14,13,17,23,14,2,
                0,0,10,15,13,12,9,11,5]
        lst_e = lst_e[:-1]
    elif subj == '51-009':
        lst_e = ['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C2','C3','C4','C5','C6','C7','C8','C9','C10', 
               'B3','B4','B5','B6','B7','B8','B9','B10', 
               'A3','A4','A5','A6','A7','A8','A9','A10']
        lst_d = [0] * len(lst_e)
    else:
        lst_e = ['F1','F2','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C3','C4','C5','C6','C7','C8','C9','C10', 
               'B1','B3','B4','B5','B6','B7','B8','B9','B10', 
               'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10']
        lst_d = [0] * len(lst_e)
    lst_s = [subj] * len(lst_e)
    
    df_o = df_o[df_o.electrode1.isin(lst_e)]
    
    df_o['distance_to_implant'] = lst_d
    df_o['subject'] = lst_s
    
    df = pd.concat([df, df_o])
data_single = data_single.merge(df, how = 'inner', on = ['electrode1','subject'])


index_lst = []
for i in range(len(data_double)):
    if not (data_single[(data_single['subject'] == data_double.subject[i]) & (data_single['electrode1'] == data_double.electrode1[i]) & (data_single['freq'] == data_double.freq[i]) & (data_single['amp1'] == data_double.amp1[i])].empty):
        if not (data_single[(data_single['subject'] == data_double.subject[i]) & (data_single['electrode1'] == data_double.electrode2[i]) & (data_single['freq'] == data_double.freq[i]) & (data_single['amp1'] == data_double.amp1[i])].empty):
            index_lst.append(i)

In [ ]:
label_list = ['MultiElectrode']
df_AllSub = pd.DataFrame()

for label in range(1):
    data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
    data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
    lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
    lst = lst.drop_duplicates().reset_index(drop=True)

    double = []
    single_dtf = []
    single_dti = []
    lst_subject = []
        
    distance = []
    amp = []
    freq = []
    between = []
    along_1 = []
    along_2 = []
    between_1 = []
    between_2 = []
    
    along = []
    electrode = []
    for i in range(len(lst)):
        electrode1 = lst.electrode1[i]
        electrode2 = lst.electrode2[i]
        temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
        if not temp1.empty and not temp2.empty and not double_temp.empty:
            count = 0
            for j in range(len(double_temp)):
                count += double_temp['num_regions'][j]
            double.append(count/len(double_temp))
            single_dtf.append((temp1.distance_to_fovea[0] + temp2.distance_to_fovea[0])/2)
            single_dti.append((temp1.distance_to_implant[0] + temp2.distance_to_implant[0])/2)
            lst_subject.append(temp1.subject[0])
                
            subject = shapes.subject_params[temp1.subject[0]]
            implant,model = shapes.model_from_params(subject)
                
            e1_x = implant[electrode1].x
            e1_y = implant[electrode1].y
            e2_x = implant[electrode2].x
            e2_y = implant[electrode2].y
                
            distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
                
            bundle = model.grow_axon_bundles()

            curve  = model.find_closest_axon(bundle, xret=e2_x, yret=e2_y)
            line = geom.LineString(curve)
            point1 = geom.Point(e1_x,e1_y)
            between1 = point1.distance(line)
            
            point_on_line1 = line.interpolate(line.project(point1))
            point2 = geom.Point(e2_x,e2_y)
            point_on_line2 = line.interpolate(line.project(point2))

            if point_on_line1.x < point_on_line2.x:
                points = np.array([[point_on_line1.x,point_on_line1.y], [point_on_line2.x,point_on_line2.y]])
            else:
                points = np.array([[point_on_line2.x,point_on_line2.y],[point_on_line1.x,point_on_line1.y]])
            curve = curve[::-1]
            start = np.searchsorted(curve[:,0], points[0,0])
            end = np.searchsorted(curve[:,0], points[1,0])
            along1 = curve_length(curve[start:end])
             
            curve  = model.find_closest_axon(bundle, xret=e1_x, yret=e1_y)
            line = geom.LineString(curve)
            point2 = geom.Point(e2_x,e2_y)
            between2 = point2.distance(line)
                
            point_on_line2 = line.interpolate(line.project(point2))
            point1 = geom.Point(e1_x,e1_y)
            point_on_line1 = line.interpolate(line.project(point1))
            
            if point_on_line2.x < point_on_line1.x:
                points = np.array([[point_on_line2.x,point_on_line2.y], [point_on_line1.x,point_on_line1.y]])
            else:
                points = np.array([[point_on_line1.x,point_on_line1.y],[point_on_line2.x,point_on_line2.y]])
            curve = curve[::-1]
            start = np.searchsorted(curve[:,0], points[0,0])
            end = np.searchsorted(curve[:,0], points[1,0])
            along2 = curve_length(curve[start:end])

            between.append(mean([between1,between2]))
            along.append(mean([along1,along2]))
                
                
        
            amp.append(lst.amp1[i])
            freq.append(lst.freq[i])

    df_investigate = pd.DataFrame({
                                       'double':double, 
                                       'freq':freq,
                                       'amp':amp,
                                        'distance':distance,
                                        'between':between,
                                        'along':along,
            
                                       'single_dtf':single_dtf,
                                       'single_dti':single_dti,
                                       'subject':lst_subject
                                      })
    for subj in ['12-005', '51-009', '52-001']:
        df_investigate_temp = df_investigate[df_investigate.subject == subj]
        filename = subj + label_list[label]
        df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
    filename = label_list[label]
    df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
    
df_AllSub = df_investigate.copy()

In [ ]:
copy = df_investigate.copy()
df_s = pd.DataFrame({})
for subj in ['12-005','51-009','52-001','All']:
    if subj=='All':
        df_t = df_AllSub.copy()
    else:   
        df_t = copy[copy.subject == subj]
    df_t['amp'] = (df_t['amp'] - df_t['amp'].mean()) / df_t['amp'].std()
    df_t['single_dti'] = (df_t['single_dti'] - df_t['single_dti'].mean()) / df_t['single_dti'].std()
    df_t['single_dtf'] = (df_t['single_dtf'] - df_t['single_dtf'].mean()) / df_t['single_dtf'].std()
    df_t['between'] = (df_t['between'] - df_t['between'].mean()) / df_t['between'].std()
    df_t['along'] = (df_t['along'] - df_t['along'].mean()) / df_t['along'].std()
    
    if subj == 'All':  
        df_AllSub = df_t.copy()
    else:
        df_s = pd.concat([df_s, df_t])
        df_s = df_s.fillna(0)
        
df_investigate = df_s.copy().reset_index(drop=True)

In [ ]:
fig, axs= plt.subplots(ncols=5, figsize=(35, 7))
column_lst = ['amp','single_dtf', 'single_dti','between','along']
name_lst = ['Amplitude','Foveal Eccentricity', 'Electrode-Implant Distance','Between-Axonal Distance','Along-Axonal Distance']
for dv in range(5):
    reg = LinearRegression().fit(df_investigate[column_lst[:dv] + column_lst[dv+1 :]], df_investigate[['double']])
    y_predicted = reg.predict(df_investigate[column_lst[:dv] + column_lst[dv+1 :]])
    y = df_investigate[['double']]-y_predicted

    reg = LinearRegression().fit(df_investigate[column_lst[:dv] + column_lst[dv+1 :]], df_investigate[column_lst[dv]])
    y_predicted = reg.predict(df_investigate[column_lst[:dv] + column_lst[dv+1 :]])
    x = df_investigate[column_lst[dv]]-y_predicted
    df_investigate['x'] = x
    df_investigate['y'] = y
    subject = ['12-005','51-009','52-001']
    marker_lst = ['o','s','^']
    color_lst = ['#1E88E5', '#FFC107','#004D40']
    for i in range(len(subject)):
        temp = df_investigate[df_investigate.subject == subject[i]]
        axs[dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.6,markersize=13)
    axs[dv].legend(subject, loc='upper right',prop={'size': 15})
    reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
    y_pred = reg.predict(np.array(x).reshape(-1,1))
    axs[dv].plot(x, y_pred,'-', color="black",linewidth=2)
    axs[dv].set(xlabel = name_lst[dv], ylabel = 'Number of Phosphenes',ylim=(-0.75,1.6))
    for item in ([axs[dv].xaxis.label, axs[dv].yaxis.label]):
        item.set_fontsize(20)
fig.savefig('/home/yuchen/paper/12f. Double-Electrode Number of Phosphenes.pdf', transparent=True)

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == '12-005':
        X = temp[['amp','single_dtf','single_dti','between','along']]
    elif subj == 'AllSubjects':
        temp = df_AllSub.copy()
        X = temp[['amp','single_dtf','single_dti','between','along']]
    else:
        X = temp[['amp','single_dtf','between','along']]
    y = temp.double
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\nvariance inflation factor: ')
    print([[X.columns[i],variance_inflation_factor(X.values, i)]
                          for i in range(len(X.columns))])
    print('\npartial correlation: ')
    X['double'] = temp['double']
    print(X.pcorr())
    print(' ')

## 2. pair_electrode shape = single_electrode shape + single_electrode shape

In [ ]:
# calculate shapes 
data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['12-005','51-009','52-001'],stim_class=None)


result = pd.DataFrame({})
for i in range(len(data)):
    label_img = skimage.measure.label(data['image'][i]>0)
    regions = regionprops(label_img)
    props = regionprops_table(label_img, properties=('centroid',
                                                     'orientation',
                                                     'major_axis_length',
                                                     'minor_axis_length',
                                                     'area',
                                                    'eccentricity',
                                                    'perimeter'))
    df = pd.DataFrame(props).astype('object')
    df.at[0,'centroid-0'] = df.iloc[:, 0].tolist()  # store centroid-x
    df.at[0,'centroid-1'] = df.iloc[:, 1].tolist()  # store centroid-y
    df.at[0,'orientation'] = df.iloc[:, 2].tolist()  # store orientation
    df.at[0,'major_axis_length'] = df.iloc[:, 3].tolist()  # major
    df.at[0,'minor_axis_length'] = df.iloc[:, 4].tolist()  # minor
    df.at[0,'area'] = df.iloc[:, 5].tolist()  # area
    df.at[0,'eccentricity'] = df.iloc[:, 6].tolist()  
    df.at[0,'perimeter'] = df.iloc[:, 7].tolist()  
    result = pd.concat([result, df.iloc[:1,:]],axis=0)

result = result.rename(columns={"area":"size", "orientation":"orientation_new", "eccentricity":"eccentricity_new" })
data = pd.concat([data,result.reset_index(drop=True)],axis=1)
data_copy = data 

In [ ]:
data = data_copy.copy()

data_double = data[(data['electrode2'] != str())].reset_index(drop=True)
data_single = data[(data['electrode2'] == str())].reset_index(drop=True)
data_single['stim_class'] ='SingleElectrode'
df = data_single[['subject', 'electrode1', 'freq', 'amp1']].drop_duplicates().reset_index(drop=True)
lst = []
lst_size = []
lst_major = []
lst_minor = []
lst_eccentricity = []
lst_orientation = []
lst_perimeter = []

for i in range(len(df)):
    df_temp = data_single[(data_single['subject'] == df.subject[i]) & (data_single['electrode1'] == df.electrode1[i]) & (data_single['freq'] == df.freq[i]) & (data_single['amp1'] == df.amp1[i])].reset_index(drop=True)
    ratio = len(df_temp[df_temp['num_regions']>1]) /len(df_temp)
    lst.append(ratio)
    if (ratio > 0.0) & (ratio <= 0.5):
        df_temp_1 = df_temp[df_temp.num_regions == 1]
        avg_size_1 = df_temp_1[['area']].mean()['area']
        avg_orientation_1 = df_temp_1[['orientation']].mean()['orientation']
        avg_eccentricity_1 = df_temp_1[['eccentricity']].mean()['eccentricity']
        df_temp_2 = df_temp[df_temp.num_regions > 1].reset_index(drop=True)
        for item in range(len(df_temp_2)):
            phosphene_id = 0
            distance = 9999999
            for phosphene_id_temp in range(df_temp_2.num_regions[item]):
                distance_temp = (float(df_temp_2['orientation_new'][item][phosphene_id_temp]) - avg_orientation_1)**2 + (float(df_temp_2['size'][item][phosphene_id_temp]) - avg_size_1)**2 + (float(df_temp_2['eccentricity_new'][item][phosphene_id_temp]) - avg_eccentricity_1)**2
                if distance_temp < distance:
                    distance = distance_temp
                    phosphene_id = phosphene_id_temp
            df_temp_2.at[item, 'size'] = [df_temp_2['size'][item][phosphene_id]]
            df_temp_2.at[item, 'major_axis_length'] = [df_temp_2['major_axis_length'][item][phosphene_id]]
            df_temp_2.at[item, 'minor_axis_length'] = [df_temp_2['minor_axis_length'][item][phosphene_id]]
            df_temp_2.at[item, 'orientation_new'] = [df_temp_2['orientation_new'][item][phosphene_id]]
            df_temp_2.at[item, 'eccentricity_new'] = [df_temp_2['eccentricity_new'][item][phosphene_id]]
            df_temp_2.at[item, 'perimeter'] = [df_temp_2['perimeter'][item][phosphene_id]]
            
            df_temp = pd.concat([df_temp_1, df_temp_2]).reset_index(drop=True)
        
        
    count_size = 0
    count_major = 0
    count_minor = 0
    count_eccentricity = 0
    count_orientation = 0
    count_perimeter = 0
    
    count_length = len(df_temp)
    for j in range(count_length):
        count_size += df_temp['size'][j][0]
        count_major += df_temp['major_axis_length'][j][0]
        count_minor += df_temp['minor_axis_length'][j][0]
        count_eccentricity += df_temp['eccentricity_new'][j][0]
        count_orientation += df_temp['orientation_new'][j][0]
        count_perimeter += df_temp['perimeter'][j][0]
        
        
    lst_size.append(count_size/count_length)
    lst_major.append(count_major/count_length)
    lst_minor.append(count_minor/count_length)
    lst_eccentricity.append(count_eccentricity/count_length)
    lst_orientation.append(count_orientation/count_length)
    lst_perimeter.append(count_perimeter/count_length)
    
df['helper'] = lst
df['avg_size'] = lst_size
df['avg_major'] = lst_major
df['avg_minor'] = lst_minor
df['avg_eccentricity'] = lst_eccentricity
df['avg_orientation'] = lst_orientation
df['avg_perimeter'] = lst_perimeter

df = df[df['helper'] <= 0.5].reset_index(drop=True)
data_single = df


index_lst = []
for i in range(len(data_double)):
    if not (data_single[(data_single['subject'] == data_double.subject[i]) & (data_single['electrode1'] == data_double.electrode1[i]) & (data_single['freq'] == data_double.freq[i]) & (data_single['amp1'] == data_double.amp1[i])].empty):
        if not (data_single[(data_single['subject'] == data_double.subject[i]) & (data_single['electrode1'] == data_double.electrode2[i]) & (data_single['freq'] == data_double.freq[i]) & (data_single['amp1'] == data_double.amp1[i])].empty):
            index_lst.append(i)

In [ ]:
subject = ['12-005','51-009','52-001']
df_dti = pd.DataFrame({})
for subj in subject:
    lst_e = []
    lst_dtf = []

    s = shapes.subject_params[subj]
    implant,model = shapes.model_from_params(s)
    for i in string.ascii_uppercase[0:6]: 
        for j in range(1,11):
            electrode = i + str(j)
            lst_e.append(electrode)
            lst_dtf.append(math.sqrt(implant[electrode].x**2 +implant[electrode].y**2 ))

    df_o = pd.DataFrame(lst_e, columns=['electrode1'])
    df_o['distance_to_fovea'] = lst_dtf

    if subj == '12-005':
        lst_d = [0,0,0,2,7,7,0,0,5,0,
                    0,0,4,11,13,15,15,3,7,4,
                    0,0,15,16,17,17,19,16,9,4,
                    0,0,16,19,15,17,22,25,13,10,
                    0,0,8,15,14,13,17,23,14,2,
                    0,0,10,15,13,12,9,11,5]
        lst_e = lst_e[:-1]
    elif subj == '51-009':
        lst_e = ['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10',
               'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
               'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
               'C2','C3','C4','C5','C6','C7','C8','C9','C10', 
               'B3','B4','B5','B6','B7','B8','B9','B10', 
               'A3','A4','A5','A6','A7','A8','A9','A10']
        lst_d = [0] * len(lst_e)
    else:
        lst_e = ['F1','F2','F4','F5','F6','F7','F8','F9','F10',
                   'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10', 
                   'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10', 
                   'C3','C4','C5','C6','C7','C8','C9','C10', 
                   'B1','B3','B4','B5','B6','B7','B8','B9','B10', 
                   'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10']
        lst_d = [0] * len(lst_e)
    lst_s = [subj] * len(lst_e)

    df_o = df_o[df_o.electrode1.isin(lst_e)]
    df_o['distance_to_implant'] = lst_d
    df_o['subject'] = lst_s
    df_dti = pd.concat([df_dti, df_o])
df_dti = df_dti.reset_index(drop=True)

data_single = data_single.merge(df_dti, how = 'inner', on = ['electrode1','subject'])

### size

In [ ]:
label_list = ['MultiElectrode']
for label in range(1):
    data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
    data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
    lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
    lst = lst.drop_duplicates().reset_index(drop=True)
    double = []
    single = []
    single1 = []
    single2 = []
    lst_subject = []
        
    distance = []
    distance_tan = []
    distance_per = []
    same_side = []   # check if both electrodes are at the same side of raphe, 1 for true, 0 for false
    within_range = []  # check if electrode is within 2mm of fovea, 1 for true, 0 for false
    between = []
    along = []
    amp = []
    freq = []
    electrode = []
    inside = []
    single_dti = []
    single_dtf=[]
    for i in range(len(lst)):
        electrode1 = lst.electrode1[i]
        electrode2 = lst.electrode2[i]
        temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
        if not temp1.empty and not temp2.empty and not double_temp.empty:
            count = 0
            for j in range(len(double_temp)):
                count += sum(double_temp['size'][j])
            double.append(count/len(double_temp))
            single.append(temp1.avg_size[0] + temp2.avg_size[0])
            single1.append(temp1.avg_size[0])
            single2.append(temp2.avg_size[0])
            
            lst_subject.append(temp1.subject[0])
                
            subject = shapes.subject_params[temp1.subject[0]]
            implant,model = shapes.model_from_params(subject)
                
            e1_x = implant[electrode1].x
            e1_y = implant[electrode1].y
            e2_x = implant[electrode2].x
            e2_y = implant[electrode2].y
            
            single_dti.append(mean([df_dti[(df_dti.electrode1 == lst.electrode1[i]) & 
                                           (df_dti.subject == lst.subject[i])].distance_to_implant.tolist()[0], 
                                    df_dti[(df_dti.electrode1 == lst.electrode2[i]) & 
                                           (df_dti.subject == lst.subject[i])].distance_to_implant.tolist()[0]]))
            
            
            distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
            single_dtf.append(mean([math.sqrt(e1_x**2 + e1_y**2), math.sqrt(e2_x**2 + e2_y**2)]))
            
            amp.append(lst.amp1[i])
            freq.append(lst.freq[i])
    
    
    
    df_investigate = pd.DataFrame({    'single':single,
                                   'single1':single1,
                                   'single2':single2,
                                       'double':double, 
                                   'single_dti':single_dti,
                                   'single_dtf':single_dtf,

                                   'distance':distance,
                                       'subject':lst_subject,
                                   'amp':amp,
                                   'freq':freq
                                      })

    temp = df_investigate[(df_investigate.amp == 2) & (df_investigate.freq == 20) & 
                              (df_investigate.single_dti == 0) & 
                              (df_investigate.single_dtf > 2950) & (df_investigate.single_dtf < 3450)]
    temp = temp.groupby(['amp','freq','subject']).mean().reset_index()
    df = pd.DataFrame({})
    for subj in temp.subject:
        df_investigate_temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
        df_investigate_temp['single'] = df_investigate_temp['single'] / temp[temp.subject == subj].iloc[0]['single']
        df_investigate_temp['double'] = df_investigate_temp['double'] / temp[temp.subject == subj].iloc[0]['double']
        df = pd.concat([df, df_investigate_temp])
    df_investigate = df.copy()

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == 'AllSubjects':
        temp = df_investigate
    X = temp[['single']]
    y = temp.double
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\npartial correlation: ')
    print(temp[['single','double']].pcorr())
    print(' ')

### major

In [ ]:
label_list = ['MultiElectrode']
for label in range(1):
    data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
    data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
    lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
    lst = lst.drop_duplicates().reset_index(drop=True)
    double = []
    single = []
    single1 = []
    single2 = []
    lst_subject = []
        
    distance = []
    distance_tan = []
    distance_per = []
    same_side = []   # check if both electrodes are at the same side of raphe, 1 for true, 0 for false
    within_range = []  # check if electrode is within 2mm of fovea, 1 for true, 0 for false
    between = []
    along = []
    amp = []
    freq = []
    electrode = []
    inside = []
    single_dti = []
    single_dtf=[]
    for i in range(len(lst)):
        electrode1 = lst.electrode1[i]
        electrode2 = lst.electrode2[i]
        temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
        if not temp1.empty and not temp2.empty and not double_temp.empty:
            count = 0
            for j in range(len(double_temp)):
                count += sum(double_temp['major_axis_length'][j])
            double.append(count/len(double_temp))
            single.append(temp1.avg_major[0] + temp2.avg_major[0])
            single1.append(temp1.avg_major[0])
            single2.append(temp2.avg_major[0])
            
            lst_subject.append(temp1.subject[0])
                
            subject = shapes.subject_params[temp1.subject[0]]
            implant,model = shapes.model_from_params(subject)
                
            e1_x = implant[electrode1].x
            e1_y = implant[electrode1].y
            e2_x = implant[electrode2].x
            e2_y = implant[electrode2].y
            
            single_dti.append(mean([df_dti[(df_dti.electrode1 == lst.electrode1[i]) & 
                                           (df_dti.subject == lst.subject[i])].distance_to_implant.tolist()[0], 
                                    df_dti[(df_dti.electrode1 == lst.electrode2[i]) & 
                                           (df_dti.subject == lst.subject[i])].distance_to_implant.tolist()[0]]))
            
            distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
            single_dtf.append(mean([math.sqrt(e1_x**2 + e1_y**2), math.sqrt(e2_x**2 + e2_y**2)]))
            
            amp.append(lst.amp1[i])
            freq.append(lst.freq[i])
    
    
    
    df_investigate = pd.DataFrame({    'single':single,
                                   'single1':single1,
                                   'single2':single2,
                                       'double':double, 
                                   'single_dti':single_dti,
                                   'single_dtf':single_dtf,

                                   'distance':distance,
                                       'subject':lst_subject,
                                   'amp':amp,
                                   'freq':freq
                                      })

    temp = df_investigate[(df_investigate.amp == 2) & (df_investigate.freq == 20) & 
                              (df_investigate.single_dti == 0) & 
                              (df_investigate.single_dtf > 2950) & (df_investigate.single_dtf < 3450)]
    temp = temp.groupby(['amp','freq','subject']).mean().reset_index()
    df = pd.DataFrame({})
    for subj in temp.subject:
        df_investigate_temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
        df_investigate_temp['single'] = df_investigate_temp['single'] / temp[temp.subject == subj].iloc[0]['single']
        df_investigate_temp['double'] = df_investigate_temp['double'] / temp[temp.subject == subj].iloc[0]['double']
        df = pd.concat([df, df_investigate_temp])
    df_investigate = df.copy()

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == 'AllSubjects':
        temp = df_investigate
    X = temp[['single']]
    y = temp.double
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\npartial correlation: ')
    print(temp[['single','double']].pcorr())
    print(' ')

### minor

In [ ]:
label_list = ['MultiElectrode']
for label in range(1):
    data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
    data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
    lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
    lst = lst.drop_duplicates().reset_index(drop=True)
    double = []
    single = []
    single1 = []
    single2 = []
    lst_subject = []
        
    distance = []
    distance_tan = []
    distance_per = []
    same_side = []   # check if both electrodes are at the same side of raphe, 1 for true, 0 for false
    within_range = []  # check if electrode is within 2mm of fovea, 1 for true, 0 for false
    between = []
    along = []
    amp = []
    freq = []
    electrode = []
    inside = []
    single_dti = []
    single_dtf=[]
    for i in range(len(lst)):
        electrode1 = lst.electrode1[i]
        electrode2 = lst.electrode2[i]
        temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
        if not temp1.empty and not temp2.empty and not double_temp.empty:
            count = 0
            for j in range(len(double_temp)):
                count += sum(double_temp['minor_axis_length'][j])
            double.append(count/len(double_temp))
            single.append(temp1.avg_minor[0] + temp2.avg_minor[0])
            single1.append(temp1.avg_minor[0])
            single2.append(temp2.avg_minor[0])
            
            lst_subject.append(temp1.subject[0])
                
            subject = shapes.subject_params[temp1.subject[0]]
            implant,model = shapes.model_from_params(subject)
                
            e1_x = implant[electrode1].x
            e1_y = implant[electrode1].y
            e2_x = implant[electrode2].x
            e2_y = implant[electrode2].y
            
            amp.append(lst.amp1[i])
            freq.append(lst.freq[i])
            
            single_dti.append(mean([df_dti[(df_dti.electrode1 == lst.electrode1[i]) & 
                                           (df_dti.subject == lst.subject[i])].distance_to_implant.tolist()[0], 
                                    df_dti[(df_dti.electrode1 == lst.electrode2[i]) & 
                                           (df_dti.subject == lst.subject[i])].distance_to_implant.tolist()[0]]))
                
            
            distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
            single_dtf.append(mean([math.sqrt(e1_x**2 + e1_y**2), math.sqrt(e2_x**2 + e2_y**2)]))
    
    
    df_investigate = pd.DataFrame({    'single':single,
                                   'single1':single1,
                                   'single2':single2,
                                       'double':double, 
                                   'single_dti':single_dti,
                                   'single_dtf':single_dtf,

                                   'distance':distance,
                                       'subject':lst_subject,
                                   'amp':amp,
                                   'freq':freq
                                      })

    temp = df_investigate[(df_investigate.amp == 2) & (df_investigate.freq == 20) & 
                              (df_investigate.single_dti == 0) & 
                              (df_investigate.single_dtf > 2950) & (df_investigate.single_dtf < 3450)]
    temp = temp.groupby(['amp','freq','subject']).mean().reset_index()
    df = pd.DataFrame({})
    for subj in temp.subject:
        df_investigate_temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
        df_investigate_temp['single'] = df_investigate_temp['single'] / temp[temp.subject == subj].iloc[0]['single']
        df_investigate_temp['double'] = df_investigate_temp['double'] / temp[temp.subject == subj].iloc[0]['double']
        df = pd.concat([df, df_investigate_temp])
    df_investigate = df.copy()

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == 'AllSubjects':
        temp = df_investigate
    X = temp[['single']]
    y = temp.double
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\npartial correlation: ')
    print(temp[['single','double']].pcorr())
    print(' ')

### perimeter

In [ ]:
label_list = ['MultiElectrode']
for label in range(1):
    data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
    data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
    lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
    lst = lst.drop_duplicates().reset_index(drop=True)
    double = []
    single = []
    single1 = []
    single2 = []
    lst_subject = []
        
    distance = []
    distance_tan = []
    distance_per = []
    between = []
    along = []
    same_side = []
    amp = []
    freq = []
    inside = []
    electrode = []
    single_dti = []
    single_dtf=[]
    for i in range(len(lst)):
        electrode1 = lst.electrode1[i]
        electrode2 = lst.electrode2[i]
        temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
        if not temp1.empty and not temp2.empty and not double_temp.empty:
            count = 0
            for j in range(len(double_temp)):
                count += sum(double_temp['perimeter'][j])
            double.append(count/len(double_temp))
            single.append(temp1.avg_perimeter[0] + temp2.avg_perimeter[0])
            single1.append(temp1.avg_perimeter[0])
            single2.append(temp2.avg_perimeter[0])
            
            lst_subject.append(temp1.subject[0])
                
            subject = shapes.subject_params[temp1.subject[0]]
            implant,model = shapes.model_from_params(subject)
                
            e1_x = implant[electrode1].x
            e1_y = implant[electrode1].y
            e2_x = implant[electrode2].x
            e2_y = implant[electrode2].y
                
            amp.append(lst.amp1[i])
            freq.append(lst.freq[i])
            
            single_dti.append(mean([df_dti[(df_dti.electrode1 == lst.electrode1[i]) & 
                                           (df_dti.subject == lst.subject[i])].distance_to_implant.tolist()[0], 
                                    df_dti[(df_dti.electrode1 == lst.electrode2[i]) & 
                                           (df_dti.subject == lst.subject[i])].distance_to_implant.tolist()[0]]))
                
            
            distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
            single_dtf.append(mean([math.sqrt(e1_x**2 + e1_y**2), math.sqrt(e2_x**2 + e2_y**2)]))
    
    
    df_investigate = pd.DataFrame({    'single':single,
                                   'single1':single1,
                                   'single2':single2,
                                       'double':double, 
                                   'single_dti':single_dti,
                                   'single_dtf':single_dtf,

                                   'distance':distance,
                                       'subject':lst_subject,
                                   'amp':amp,
                                   'freq':freq
                                      })

    temp = df_investigate[(df_investigate.amp == 2) & (df_investigate.freq == 20) & 
                              (df_investigate.single_dti == 0) & 
                              (df_investigate.single_dtf > 2950) & (df_investigate.single_dtf < 3450)]
    temp = temp.groupby(['amp','freq','subject']).mean().reset_index()
    df = pd.DataFrame({})
    for subj in temp.subject:
        df_investigate_temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
        df_investigate_temp['single'] = df_investigate_temp['single'] / temp[temp.subject == subj].iloc[0]['single']
        df_investigate_temp['double'] = df_investigate_temp['double'] / temp[temp.subject == subj].iloc[0]['double']
        df = pd.concat([df, df_investigate_temp])
    df_investigate = df.copy()

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == 'AllSubjects':
        temp = df_investigate
    X = temp[['single']]
    y = temp.double
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\npartial correlation: ')
    print(temp[['single','double']].pcorr())
    print(' ')

### orientation

In [ ]:
label_list = ['MultiElectrode']
for label in range(1):
    data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
    data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
    lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
    lst = lst.drop_duplicates().reset_index(drop=True)
    double = []
    single = []
    single1 = []
    single2 = []
    lst_subject = []
    between = []
    along = []
    distance = []
    distance_tan = []
    distance_per = []
    same_side = []
    inside = []
    amp = []
    freq = []
    electrode = []
    for i in range(len(lst)):
        electrode1 = lst.electrode1[i]
        electrode2 = lst.electrode2[i]
        temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
        if not temp1.empty and not temp2.empty and not double_temp.empty:
            count = 0
            for j in range(len(double_temp)):
                count += mean(double_temp['orientation_new'][j])
            double.append(count/len(double_temp))
            single.append(mean([temp1.avg_orientation[0], temp2.avg_orientation[0]]))
            single1.append(temp1.avg_orientation[0])
            single2.append(temp2.avg_orientation[0])
            
            lst_subject.append(temp1.subject[0])
                
            subject = shapes.subject_params[temp1.subject[0]]
            implant,model = shapes.model_from_params(subject)
                
            e1_x = implant[electrode1].x
            e1_y = implant[electrode1].y
            e2_x = implant[electrode2].x
            e2_y = implant[electrode2].y

            distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
            
            amp.append(lst.amp1[i])
            freq.append(lst.freq[i])

    df_investigate = pd.DataFrame({    'single':single,
                                   'single1':single1,
                                   'single2':single2,
                                       'double':double, 
                                   'distance':distance,
                                       'subject':lst_subject,
                                   'amp':amp,
                                   'freq':freq
                                      })
    for subj in ['12-005', '51-009', '52-001']:
        df_investigate_temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
        filename = subj + label_list[label] 
        df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
    filename = label_list[label]
    df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == 'AllSubjects':
        temp = df_investigate
    X = temp[['single']]
    y = temp.double
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\npartial correlation: ')
    print(temp[['single','double']].pcorr())
    print(' ')

### eccentricity

In [ ]:
label_list = ['SpatialSummation']
for label in range(1):
    data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
    data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
    lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
    lst = lst.drop_duplicates().reset_index(drop=True)
    double = []
    single = []
    single1 = []
    single2 = []
    same_side = []
    lst_subject = []
        
    distance = []
    distance_per = []
    distance_tan = []
    amp = []
    freq = []
    between = []
    along = []
    inside = []
    
    electrode = []
    for i in range(len(lst)):
        electrode1 = lst.electrode1[i]
        electrode2 = lst.electrode2[i]
        temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
        if not temp1.empty and not temp2.empty and not double_temp.empty:
            count = 0
            for j in range(len(double_temp)):
                count += mean(double_temp['eccentricity_new'][j])
            double.append(count/len(double_temp))
            single.append(mean([temp1.avg_eccentricity[0],temp2.avg_eccentricity[0]]))
            single1.append(temp1.avg_eccentricity[0])
            single2.append(temp2.avg_eccentricity[0])
            
            lst_subject.append(temp1.subject[0])
                
            subject = shapes.subject_params[temp1.subject[0]]
            implant,model = shapes.model_from_params(subject)
                
            e1_x = implant[electrode1].x
            e1_y = implant[electrode1].y
            e2_x = implant[electrode2].x
            e2_y = implant[electrode2].y
            
            # if abs(e1_x) < 2750 and abs(e1_y) < 2750 and abs(e2_x) <2750 and abs(e2_y) < 2750:
            #     inside.append(1)
            # else:
            #     inside.append(0)

            distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
            
#             bundle = model.grow_axon_bundles()
#             curve  = model.find_closest_axon(bundle, xret=e2_x, yret=e2_y)
#             line = geom.LineString(curve)
#             point = geom.Point(e1_x,e1_y)
#             between1 = point.distance(line)
#             point_on_line = line.interpolate(line.project(point))

#             if point_on_line.x < e2_x:
#                 points = np.array([[point_on_line.x,point_on_line.y], [e2_x,e2_y]])
#             else:
#                 points = np.array([[e2_x,e2_y],[point_on_line.x,point_on_line.y]])
#             start = np.searchsorted(curve[:,0], points[0,0])
#             curve = np.insert(curve, start, points[0], axis=0)
#             end = np.searchsorted(curve[:,0], points[1,0])
#             curve = np.insert(curve, end, points[1], axis=0)

#             along1 = curve_length(curve[start:end+1])

#             curve  = model.find_closest_axon(bundle, xret=e1_x, yret=e1_y)
#             line = geom.LineString(curve)
#             point = geom.Point(e2_x,e2_y)
#             between2 = point.distance(line)
#             point_on_line = line.interpolate(line.project(point))

#             if point_on_line.x < e1_x:
#                 points = np.array([[point_on_line.x,point_on_line.y], [e1_x,e1_y]])
#             else:
#                 points = np.array([[e1_x,e1_y],[point_on_line.x,point_on_line.y]])
#             start = np.searchsorted(curve[:,0], points[0,0])
#             curve = np.insert(curve, start, points[0], axis=0)
#             end = np.searchsorted(curve[:,0], points[1,0])
#             curve = np.insert(curve, end, points[1], axis=0)
#             along2 = curve_length(curve[start:end+1])
            
#             between.append(mean([between1, between2]))
#             along.append(mean([along1, along2]))
            
            amp.append(lst.amp1[i])
            freq.append(lst.freq[i])

    df_investigate = pd.DataFrame({    'single':single,
                                   'single1':single1,
                                   'single2':single2,
                                       'double':double, 
                                   'distance':distance,
                                   # 'between':between,
                                   # 'along':along,
                                   # 'inside':inside,
                                       'subject':lst_subject,
                                   'amp':amp,
                                   'freq':freq
                                      })
    for subj in ['12-005', '51-009', '52-001']:
        df_investigate_temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
        filename = subj + label_list[label] 
        df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
    filename = label_list[label]
    df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')

### phosphene number 

In [ ]:
data = data_copy

data_double = data[(data['electrode2'] != str())].reset_index(drop=True)
data_single = data[(data['electrode2'] == str())].reset_index(drop=True)
data_single['stim_class'] ='SingleElectrode'
df = data_single[['subject', 'electrode1', 'freq', 'amp1']].drop_duplicates().reset_index(drop=True)

lst = []
lst_phosphene = []

for i in range(len(df)):
    df_temp = data_single[(data_single['subject'] == df.subject[i]) & (data_single['electrode1'] == df.electrode1[i]) & (data_single['freq'] == df.freq[i]) & (data_single['amp1'] == df.amp1[i])].reset_index(drop=True)
    ratio = len(df_temp[df_temp['num_regions']>1]) /len(df_temp)
    lst.append(ratio)
    lst_phosphene.append(mean(df_temp['num_regions']))
        
df['helper'] = lst
df['phosphene'] = lst_phosphene

data_single = df


index_lst = []
for i in range(len(data_double)):
    if not (data_single[(data_single['subject'] == data_double.subject[i]) & (data_single['electrode1'] == data_double.electrode1[i]) & (data_single['freq'] == data_double.freq[i]) & (data_single['amp1'] == data_double.amp1[i])].empty):
        if not (data_single[(data_single['subject'] == data_double.subject[i]) & (data_single['electrode1'] == data_double.electrode2[i]) & (data_single['freq'] == data_double.freq[i]) & (data_single['amp1'] == data_double.amp1[i])].empty):
            index_lst.append(i)
            
label_list = ['MultiElectrode']

for label in range(1):
    data_double_temp = data_double.iloc[index_lst].reset_index(drop=True)
    data_double_temp = data_double_temp[data_double_temp['stim_class'] == label_list[label]]
    lst = data_double_temp[['electrode1','electrode2','amp1','freq','subject']]
    lst = lst.drop_duplicates().reset_index(drop=True)

    double = []
    single = []
    single1 = []
    single2 = []
    same_side = []
    distance_per = []
    distance_tan = []
    lst_subject = []
    distance = []
    electrode = []
    between = []
    along = []
    amp = []
    freq = []
    inside = []
    for i in range(len(lst)):
        electrode1 = lst.electrode1[i]
        electrode2 = lst.electrode2[i]
        temp1 = data_single[(data_single['electrode1'] == electrode1) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        temp2 = data_single[(data_single['electrode1'] == electrode2) &  (data_single['subject'] == lst.subject[i])  & (data_single['amp1'] == lst.amp1[i]) & (data_single['freq'] == lst.freq[i])].reset_index(drop=True)
        double_temp = data_double_temp[(data_double_temp['electrode1'] == electrode1) & ((data_double_temp['electrode2'] == electrode2))& (data_double_temp['subject'] == lst.subject[i]) & (data_double_temp['amp1'] == lst.amp1[i]) & (data_double_temp['freq'] == lst.freq[i])].reset_index(drop=True)
        if not temp1.empty and not temp2.empty and not double_temp.empty:
            count = 0
            for j in range(len(double_temp)):
                count += double_temp['num_regions'][j]
            double.append(count/len(double_temp))
            single.append(sum([temp1.phosphene[0], temp2.phosphene[0]]))
            single1.append(temp1.phosphene[0])
            single2.append(temp2.phosphene[0])
            
            lst_subject.append(temp1.subject[0])
                
            subject = shapes.subject_params[temp1.subject[0]]
            implant,model = shapes.model_from_params(subject)
            e1_x = implant[electrode1].x
            e1_y = implant[electrode1].y
            e2_x = implant[electrode2].x
            e2_y = implant[electrode2].y
            
#             if abs(e1_x) < 2750 and abs(e1_y) < 2750 and abs(e2_x) <2750 and abs(e2_y) < 2750:
#                 inside.append(1)
#             else:
#                 inside.append(0)

            distance.append(math.sqrt((e1_x - e2_x)**2 + (e1_y - e2_y)**2))
            
#             bundle = model.grow_axon_bundles()
#             curve  = model.find_closest_axon(bundle, xret=e2_x, yret=e2_y)
#             line = geom.LineString(curve)
#             point = geom.Point(e1_x,e1_y)
#             between1 = point.distance(line)
#             point_on_line = line.interpolate(line.project(point))

#             if point_on_line.x < e2_x:
#                 points = np.array([[point_on_line.x,point_on_line.y], [e2_x,e2_y]])
#             else:
#                 points = np.array([[e2_x,e2_y],[point_on_line.x,point_on_line.y]])
#             start = np.searchsorted(curve[:,0], points[0,0])
#             curve = np.insert(curve, start, points[0], axis=0)
#             end = np.searchsorted(curve[:,0], points[1,0])
#             curve = np.insert(curve, end, points[1], axis=0)

#             along1 = curve_length(curve[start:end+1])

#             curve  = model.find_closest_axon(bundle, xret=e1_x, yret=e1_y)
#             line = geom.LineString(curve)
#             point = geom.Point(e2_x,e2_y)
#             between2 = point.distance(line)
#             point_on_line = line.interpolate(line.project(point))

#             if point_on_line.x < e1_x:
#                 points = np.array([[point_on_line.x,point_on_line.y], [e1_x,e1_y]])
#             else:
#                 points = np.array([[e1_x,e1_y],[point_on_line.x,point_on_line.y]])
#             start = np.searchsorted(curve[:,0], points[0,0])
#             curve = np.insert(curve, start, points[0], axis=0)
#             end = np.searchsorted(curve[:,0], points[1,0])
#             curve = np.insert(curve, end, points[1], axis=0)
#             along2 = curve_length(curve[start:end+1])
            
#             between.append(mean([between1, between2]))
#             along.append(mean([along1, along2]))
            
            amp.append(lst.amp1[i])
            freq.append(lst.freq[i])

    df_investigate = pd.DataFrame({    'single':single,
                                   'single1':single1,
                                   'single2':single2,
                                       'double':double, 
                                   'distance':distance,
                                   # 'between':between,
                                   # 'along':along,
                                   # 'inside':inside,
                                       'subject':lst_subject,
                                   'amp':amp,
                                   'freq':freq
                                      })
    for subj in ['12-005', '51-009', '52-001']:
        df_investigate_temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
        filename = subj + label_list[label] 
        df_investigate_temp.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')
    filename = label_list[label]
    df_investigate.to_csv('/home/yuchen/shapes/notebooks/' + filename + '.csv')

In [ ]:
for subj in ['12-005','51-009','52-001','AllSubjects']:
    temp = df_investigate[df_investigate.subject == subj].reset_index(drop=True)
    if subj == 'AllSubjects':
        temp = df_investigate
    X = temp[['single']]
    y = temp.double
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print('subject: ' + subj)
    print(est2.summary())
    print('\ncoefficient with more digits: ')
    print(est2.params)
    print('\npartial correlation: ')
    print(temp[['single','double']].pcorr())
    print(' ')

In [ ]:
from scipy import stats
stats.ttest_ind(df1.double, df2.double, equal_var=False, alternative='two-sided')
# no difference between sequential stimulation and simultaneous stimulation in terms of phosphene numbers

In [ ]:
# In sequential stimulation
temp1 = df1[(df1.same_side == 0) & (df1.distance)]
temp2 = df1[df1.same_side == 1]
print(temp1.distance.mean())
print(temp2.distance.mean())
stats.ttest_ind(temp1.double, temp2.double, equal_var=False, alternative='greater')
# Electrodes on different side of the raphe are more likely to produce multiple phosphenes

In [ ]:
# In simultaneous stimulation
temp1 = df2[df2.same_side == 0]
temp2 = df2[df2.same_side == 1]
print(temp1.distance.mean())
print(temp2.distance.mean())
stats.ttest_ind(temp1.double, temp2.double, equal_var=False, alternative='greater')
# Electrodes on different side of the raphe are more likely to produce multiple phosphenes

In [ ]:
fig, axs= plt.subplots(ncols=3, nrows = 2, figsize=(21, 14))
column_lst = ['single', 'distance_tan','distance_per']
name_lst = ['Single Electrode Shape','Tangential Distance','Perpendicular Distance', 'Electrode-Implant Distance']
for same_side in range(2):
    df_investigate_temp = df_investigate[df_investigate.same_side == same_side]
    for dv in range(3):
        reg = LinearRegression().fit(df_investigate_temp[column_lst[:dv] + column_lst[dv+1 :]], df_investigate_temp[['double']])
        y_predicted = reg.predict(df_investigate_temp[column_lst[:dv] + column_lst[dv+1 :]])
        y = df_investigate_temp[['double']]-y_predicted

        reg = LinearRegression().fit(df_investigate_temp[column_lst[:dv] + column_lst[dv+1 :]], df_investigate_temp[column_lst[dv]])
        y_predicted = reg.predict(df_investigate_temp[column_lst[:dv] + column_lst[dv+1 :]])
        x = df_investigate_temp[column_lst[dv]]-y_predicted
        df_investigate_temp['x'] = x
        df_investigate_temp['y'] = y
        subject = ['12-005','51-009','52-001']
        marker_lst = ['o','X','s']
        color_lst = ['#1E88E5', '#FFC107','#004D40']
        for i in range(len(subject)):
            temp = df_investigate_temp[df_investigate_temp.subject == subject[i]]
            axs[same_side,dv].plot(temp['x'],temp['y'],marker_lst[i], color=color_lst[i],alpha=0.8)
        axs[same_side,dv].legend(subject, loc='upper right')
        reg = LinearRegression().fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))
        y_pred = reg.predict(np.array(x).reshape(-1,1))
        axs[same_side,dv].plot(x, y_pred,'-', color="black",linewidth=2)
        axs[same_side,dv].text(x=x.min(),y=y.max(), s='r = '+ str(np.corrcoef(df_investigate_temp.x,df_investigate_temp.y)[0,1].round(3)))
        axs[same_side,dv].set(xlabel = name_lst[dv], ylabel = 'Area (Double Electrode)')
axs[0,0].set(ylabel = 'Number of Phosphene (Double Electrode) - electrodes on the different side of raphe')
axs[1,0].set(ylabel = 'Number of Phosphene (Double Electrode) - electrodes on the same side of raphe')

## (discarded) pair_electrode shape = single_electrode shape + single_electrode shape - overlapped shape (map based on centroid)

In [ ]:
df['centroid-1'][image][]

In [ ]:
count_0 = 0
count_1 = 0
test = []
lst = []
for i in range(len(data)):
    if data.electrode2[i] == str():
        lst.append('SingleElectrode')
    else:
        lst.append(data.stim_class[i])
data['stim_class'] = lst

cate = data[['subject','amp1','freq','electrode1','electrode2','stim_class']].drop_duplicates().reset_index(drop=True)

img = []
for i in range(len(cate)):
    df = data[(data['subject'] == cate['subject'][i]) & 
              (data['amp1'] == cate['amp1'][i]) & 
              (data['electrode1'] == cate['electrode1'][i]) &
              (data['electrode2'] == cate['electrode2'][i]) & 
              (data['stim_class'] == cate['stim_class'][i])
             ].reset_index(drop=True)
    if df.empty:
        img.append(np.zeros((384, 512)))
    else:
        if df.electrode2[0] == str():  # single electrode 
            if (len(df[df['num_regions']>1])/(len(df))) <= 0.5:
                count_1 += 1
                df_1 = df[df['num_regions'] == 1].reset_index(drop=True)
                df_centroid_0_avg = 0
                df_centroid_1_avg = 0
                for item in range(len(df_1)):
                    df_centroid_0_avg = df_centroid_0_avg + df_1['centroid-0'][item][0]
                    df_centroid_1_avg = df_centroid_1_avg + df_1['centroid-1'][item][0]
                df_centroid_0_avg = df_centroid_0_avg/len(df_1)
                df_centroid_1_avg = df_centroid_1_avg/len(df_1)

                # for drawings with more than one phosphenes, select the one with centroid cloest to the averaged centroid
                df_2 = df[df['num_regions'] > 1].reset_index(drop=True)
                if len(df_2) != 0:
                    for double_number in range(len(df_2)):
                        phosphene_id = 0
                        distance = 9999999
                        for double_item in range(len(df_2['centroid-0'][double_number])):
                            distance_temp = (df_2['centroid-0'][double_number][double_item] - df_centroid_0_avg)**2 + (df_2['centroid-1'][double_number][double_item] - df_centroid_1_avg)**2
                            if distance_temp < distance:
                                distance = distance_temp
                                phosphene_id = double_item
                        df_2.at[double_number,'centroid-0'] = [df_2['centroid-0'][double_number][phosphene_id]]
                        df_2.at[double_number,'centroid-1'] = [df_2['centroid-1'][double_number][phosphene_id]]

                df = pd.concat([df_1,df_2], axis=0).reset_index(drop=True)
                image_temp = np.zeros((len(df['image'][0]),len(df['image'][0][0])))
                for image in range(len(df)):
                    image_temp = image_temp + p2p.utils.images.shift_image(df['image'][image],df_centroid_1_avg-df['centroid-1'][image][0], df_centroid_0_avg-df['centroid-0'][image][0])
                plt.figure()
                plt.imshow(image_temp)
                image_temp = image_temp / len(df)
                # for r in range(len(image_temp)):
                #     for c in range(len(image_temp[r])):
                #         if image_temp[r][c] < 0.5:
                #             image_temp[r][c] = 0
                #         else:
                #             image_temp[r][c] = 1
                img.append(image_temp)

            else:  # remove single electrode data with >=50% drawings having >1 phoephenes
                img.append(np.zeros((len(df['image'][0]),len(df['image'][0][0]))))
                count_0 += 1 
        elif df.electrode2[0] != str():  # double electrode
            phosphene1 = np.mean([p2p.utils.images.center_image(skimage.measure.label(image) == 1) for image in df.image], axis=0)
            phosphene2 = np.mean([p2p.utils.images.center_image(skimage.measure.label(image) == 2) for image in df.image], axis=0)
            phosphene1_centroid = np.mean([regionprops(skimage.measure.label(image>0))[0].centroid for image in df.image], axis=0)
            if sum(sum(phosphene2)) != 0:
                count = (0,0)
                phosphene2_centroid = 0
                for image in df.image:
                    if len(regionprops(skimage.measure.label(image>0))) >= 2:
                        count = (count[0] + regionprops(skimage.measure.label(image>0))[1].centroid[0],count[1] + regionprops(skimage.measure.label(image>0))[1].centroid[1]) 
                        phosphene2_centroid += 1
                phosphene2_centroid = (count[0] / phosphene2_centroid,count[1] / phosphene2_centroid)
            y_center, x_center = phosphene1.shape[0] / 2, phosphene1.shape[1] / 2
            phosphene1 = p2p.utils.images.shift_image(phosphene1, phosphene1_centroid[1] - x_center, phosphene1_centroid[0] - y_center)
            if sum(sum(phosphene2)) != 0:
                phosphene2 = p2p.utils.images.shift_image(phosphene2, phosphene2_centroid[1] - x_center, phosphene2_centroid[0] - y_center)
                image_temp = phosphene1 + phosphene2
            else:
                image_temp = phosphene1
            test.append(image_temp)
            # for r in range(len(image_temp)):
            #     for c in range(len(image_temp[r])):
            #         if image_temp[r][c] < 0.5:
            #             image_temp[r][c] = 0
            #         else:
            #             image_temp[r][c] = 1
            img.append(image_temp)
            
cate['avg_image'] = img

In [ ]:
cate = cate[cate.stim_class != 'SpatialSummation'].reset_index(drop=True)
cate

In [ ]:

i=20
f1 = cate[(cate.electrode1 == temp.electrode1[i]) & (cate.electrode2 == str()) & (cate.amp1 == temp.amp1[i]) & (cate.freq == temp.freq[i]) & (cate.subject == temp.subject[i])]
f2 = cate[(cate.electrode1 == temp.electrode2[i]) & (cate.electrode2 == str()) & (cate.amp1 == temp.amp1[i]) & (cate.freq == temp.freq[i]) & (cate.subject == temp.subject[i])]
temp[20:21]

In [ ]:
plt.imshow(temp.avg_image[20:21][20],cmap='gray')

In [ ]:
temp.avg_image[20:21]

In [ ]:
temp = cate[cate.electrode2 != str()].reset_index(drop=True)

for i in range(len(temp)):
    f1 = cate[(cate.electrode1 == temp.electrode1[i]) & (cate.electrode2 == str()) & (cate.amp1 == temp.amp1[i]) & (cate.freq == temp.freq[i]) & (cate.subject == temp.subject[i])]
    f2 = cate[(cate.electrode1 == temp.electrode2[i]) & (cate.electrode2 == str()) & (cate.amp1 == temp.amp1[i]) & (cate.freq == temp.freq[i]) & (cate.subject == temp.subject[i])]
    
    if not f1.empty and not f2.empty:
        ig, axs= plt.subplots(ncols=3, nrows = 1, figsize=(21, 7))
        axs[0].imshow(f1.avg_image.iloc[0],cmap = 'gray')
        axs[1].imshow(f2.avg_image.iloc[0],cmap = 'gray')       
        axs[2].imshow(temp.avg_image[i],cmap = 'gray')
        axs[0].set(title=f1.electrode1.iloc[0] + ' ' + f1.electrode2.iloc[0])
        axs[1].set(title=f2.electrode1.iloc[0] + ' ' + f2.electrode2.iloc[0])
        axs[2].set(title=temp.electrode1[i] + ' ' + temp.electrode2[i])

In [ ]:
k = data[(data.electrode1 == 'F4') & (data.electrode2 == 'F7')].reset_index(drop=True)
for i in range(len(k)):
    plt.figure()
    plt.imshow(k.image[i], cmap='gray')

In [ ]:
result = pd.DataFrame({})
cate_temp = pd.DataFrame({})
for i in range(len(cate)):
    label_img = skimage.measure.label(cate['avg_image'][i]>0)
    regions = regionprops(label_img)
    props = regionprops_table(label_img, properties=('centroid',
                                                     'orientation',
                                                     'major_axis_length',
                                                     'minor_axis_length',
                                                     'area'))
    df = pd.DataFrame(props).astype('object')
    if not df.empty:
        df.at[0,'centroid-0'] = df.iloc[:, 0].tolist()  # store centroid-x
        df.at[0,'centroid-1'] = df.iloc[:, 1].tolist()  # store centroid-y
        df.at[0,'orientation'] = df.iloc[:, 2].tolist()  # store orientation
        df.at[0,'major_axis_length'] = df.iloc[:, 3].tolist()  # major
        df.at[0,'minor_axis_length'] = df.iloc[:, 4].tolist()  # minor
        df.at[0,'area'] = df.iloc[:, 5].tolist()  # area
        result = pd.concat([result, df.iloc[:1,:]],axis=0)
        cate_temp = pd.concat([cate_temp, cate[i:i+1]],axis=0)

result = result.rename(columns={"area":"size"})
cate = pd.concat([cate_temp.reset_index(drop=True), result.reset_index(drop=True)],axis=1)

### size

In [ ]:
fig, axes = plt.subplots(ncols=3,nrows=2, figsize=(20,14))
marker_lst = ['o','^']
color_lst = ['blue','black']

label_list = ['MultiElectrode', 'SpatialSummation']

for condition in range(2):
    if condition == 1:
        cate_temp = cate[cate['electrode2'] != str()].reset_index(drop=True)
        lst = cate_temp[['subject','amp1','freq','electrode1','electrode2','stim_class']].groupby(['subject','amp1','freq','electrode1','electrode2']).count().stim_class.tolist()
        lst_2 = cate_temp[['subject','amp1','freq','electrode1','electrode2','stim_class']].groupby(['subject','amp1','freq','electrode1','electrode2']).count().index.to_frame(index=False)
        lst_2['count'] = lst
        lst_2 = lst_2[(lst_2['count'] == 2) & (lst_2.electrode2 != str()) & (lst_2.amp1 == 2.0)].reset_index(drop=True)
        cate_temp = pd.merge(cate_temp, lst_2, how = "inner", on=["electrode1", "electrode2",'subject','amp1','freq'])
    else:
        cate_temp = cate[(cate['amp1'] == 2.0)].reset_index(drop=True)
        
        
    for label in range(2):
        df_temp = cate_temp[cate_temp.stim_class == label_list[label]].reset_index(drop=True)
        double = []
        single = []
        for i in range(len(df_temp)):
            df_1 = cate[(cate['electrode1'] == df_temp.electrode1[i]) & 
             (cate['electrode2'] == str()) & 
             (cate['amp1'] == df_temp.amp1[i]) & 
             (cate['freq'] == df_temp.freq[i]) &
             (cate['subject'] == df_temp.subject[i])]
            df_2 = cate[(cate['electrode1'] == df_temp.electrode2[i]) & 
             (cate['electrode2'] == str()) & 
             (cate['amp1'] == df_temp.amp1[i]) & 
             (cate['freq'] == df_temp.freq[i]) &
             (cate['subject'] == df_temp.subject[i])]
            if df_1.size != 0 and df_2.size != 0:
                double_size = sum(df_temp['size'][i])
                single_size1 = sum(df_1['size'].tolist()[0])
                single_size2 = sum(df_2['size'].tolist()[0])
                if double_size>0 and single_size1>0 and single_size2>0:
                    double.append(double_size)
                    combined_image = df_1.avg_image.reset_index(drop=True)[0] + df_2.avg_image.reset_index(drop=True)[0]
                    label_img = skimage.measure.label(combined_image)
                    regions = regionprops(label_img)
                    props = regionprops_table(label_img, properties=('centroid',
                                                                     'orientation',
                                                                     'major_axis_length',
                                                                     'minor_axis_length',
                                                                         'area'))
                    df = pd.DataFrame(props).astype('object')
                    if not df.empty:
                        df.at[0,'centroid-0'] = df.iloc[:, 0].tolist()  # store centroid-x
                        df.at[0,'centroid-1'] = df.iloc[:, 1].tolist()  # store centroid-y
                        df.at[0,'orientation'] = df.iloc[:, 2].tolist()  # store orientation
                        df.at[0,'major_axis_length'] = df.iloc[:, 3].tolist()  # major
                        df.at[0,'minor_axis_length'] = df.iloc[:, 4].tolist()  # minor
                        df.at[0,'area'] = df.iloc[:, 5].tolist()  # area
                    
                    single.append(sum(df['area'][0]))
        lst = []
        for i in range(len(double)):
            if double[i] > 3000 or single[i] > 3000:
                lst.append(i)
        for i in range(len(lst)):
            del double[lst[-i-1]]
            del single[lst[-i-1]]   
        scatter_correlation(double,single, ax=axes[condition][label])
        scatter_correlation(double,single, ax = axes[condition][2], marker = marker_lst[label], color = color_lst[label], text = False)

axes[0][0].set(xlabel='double', ylabel='single', xlim = (0,2500), ylim = (-100,2700), title='size (simultaneous)')
axes[0][1].set(xlabel='double', ylabel='single' , xlim = (0,2500), ylim = (-100,2700),title='size (sequential)')
axes[0][2].set(xlabel='double', ylabel='single' , xlim = (0,2500), ylim = (-100,2700))
axes[1][2].set(xlabel='double', ylabel='single' , xlim = (0,2500), ylim = (-100,2700))
axes[1][0].set(xlabel='double', ylabel='single' , xlim = (0,2500), ylim = (-100,2700),title='size (simultaneous - groups with sequential data only)')
axes[1][1].set(xlabel='double', ylabel='single', xlim = (0,2500), ylim = (-100,2700), title='size (simultaneous - groups with simultaneous data only)')
axes[0][2].legend(['simultaneous','sequential'],loc='upper right')
axes[1][2].legend(['simultaneous','sequential'],loc='upper right')

### major_axis_length

In [ ]:
fig, axes = plt.subplots(ncols=3,nrows=2, figsize=(20,14))
marker_lst = ['o','^']
color_lst = ['blue','black']

label_list = ['MultiElectrode', 'SpatialSummation']

for condition in range(2):
    if condition == 1:
        cate_temp = cate[cate['electrode2'] != str()].reset_index(drop=True)
        lst = cate_temp[['subject','amp1','freq','electrode1','electrode2','stim_class']].groupby(['subject','amp1','freq','electrode1','electrode2']).count().stim_class.tolist()
        lst_2 = cate_temp[['subject','amp1','freq','electrode1','electrode2','stim_class']].groupby(['subject','amp1','freq','electrode1','electrode2']).count().index.to_frame(index=False)
        lst_2['count'] = lst
        lst_2 = lst_2[(lst_2['count'] == 2) & (lst_2.electrode2 != str()) & (lst_2.amp1 == 2.0)].reset_index(drop=True)
        cate_temp = pd.merge(cate_temp, lst_2, how = "inner", on=["electrode1", "electrode2",'subject','amp1','freq'])
    else:
        cate_temp = cate[(cate['amp1'] == 2.0)].reset_index(drop=True)
        
        
    for label in range(2):
        df_temp = cate_temp[cate_temp.stim_class == label_list[label]].reset_index(drop=True)
        double = []
        single = []
        for i in range(len(df_temp)):
            df_1 = cate[(cate['electrode1'] == df_temp.electrode1[i]) & 
             (cate['electrode2'] == str()) & 
             (cate['amp1'] == df_temp.amp1[i]) & 
             (cate['freq'] == df_temp.freq[i]) &
             (cate['subject'] == df_temp.subject[i])]
            df_2 = cate[(cate['electrode1'] == df_temp.electrode2[i]) & 
             (cate['electrode2'] == str()) & 
             (cate['amp1'] == df_temp.amp1[i]) & 
             (cate['freq'] == df_temp.freq[i]) &
             (cate['subject'] == df_temp.subject[i])]
            if df_1.size != 0 and df_2.size != 0:
                double_size = sum(df_temp['major_axis_length'][i])
                single_size1 = sum(df_1['major_axis_length'].tolist()[0])
                single_size2 = sum(df_2['major_axis_length'].tolist()[0])
                if double_size>0 and single_size1>0 and single_size2>0:
                    double.append(double_size)
                    combined_image = df_1.avg_image.reset_index(drop=True)[0] + df_2.avg_image.reset_index(drop=True)[0]
                    label_img = skimage.measure.label(combined_image)
                    regions = regionprops(label_img)
                    props = regionprops_table(label_img, properties=('centroid',
                                                                     'orientation',
                                                                     'major_axis_length',
                                                                     'minor_axis_length',
                                                                         'area'))
                    df = pd.DataFrame(props).astype('object')
                    if not df.empty:
                        df.at[0,'centroid-0'] = df.iloc[:, 0].tolist()  # store centroid-x
                        df.at[0,'centroid-1'] = df.iloc[:, 1].tolist()  # store centroid-y
                        df.at[0,'orientation'] = df.iloc[:, 2].tolist()  # store orientation
                        df.at[0,'major_axis_length'] = df.iloc[:, 3].tolist()  # major
                        df.at[0,'minor_axis_length'] = df.iloc[:, 4].tolist()  # minor
                        df.at[0,'area'] = df.iloc[:, 5].tolist()  # area
                    
                    single.append(sum(df['major_axis_length'][0]))
        lst = []
        for i in range(len(double)):
            if double[i] > 200 or single[i] > 200:
                lst.append(i)
        for i in range(len(lst)):
            del double[lst[-i-1]]
            del single[lst[-i-1]]   
        scatter_correlation(double,single, ax=axes[condition][label])
        scatter_correlation(double,single, ax = axes[condition][2], marker = marker_lst[label], color = color_lst[label], text = False)

axes[0][0].set(xlabel='double', ylabel='single', xlim = (0, 190), ylim = (0,170),  title='major_axis_length (simultaneous)')
axes[0][1].set(xlabel='double', ylabel='single' , xlim = (0, 190), ylim = (0,170), title='major_axis_length (sequential)')
axes[0][2].set(xlabel='double', ylabel='single', xlim = (0, 190), ylim = (0,170))
axes[1][2].set(xlabel='double', ylabel='single' , xlim = (0, 190), ylim = (0,170))
axes[1][0].set(xlabel='double', ylabel='single' , xlim = (0, 190), ylim = (0,170), title='major_axis_length (simultaneous - groups with sequential data only)')
axes[1][1].set(xlabel='double', ylabel='single',  xlim = (0, 190), ylim = (0,170), title='major_axis_length (simultaneous - groups with simultaneous data only)')
axes[0][2].legend(['simultaneous','sequential'],loc='upper right')
axes[1][2].legend(['simultaneous','sequential'],loc='upper right')

### minor_axis_length

In [ ]:
fig, axes = plt.subplots(ncols=3,nrows=2, figsize=(20,14))
marker_lst = ['o','^']
color_lst = ['blue','black']
label_list = ['MultiElectrode', 'SpatialSummation']

for condition in range(2):
    if condition == 1:
        cate_temp = cate[cate['electrode2'] != str()].reset_index(drop=True)
        lst = cate_temp[['subject','amp1','freq','electrode1','electrode2','stim_class']].groupby(['subject','amp1','freq','electrode1','electrode2']).count().stim_class.tolist()
        lst_2 = cate_temp[['subject','amp1','freq','electrode1','electrode2','stim_class']].groupby(['subject','amp1','freq','electrode1','electrode2']).count().index.to_frame(index=False)
        lst_2['count'] = lst
        lst_2 = lst_2[(lst_2['count'] == 2) & (lst_2.electrode2 != str()) & (lst_2.amp1 == 2.0)].reset_index(drop=True)
        cate_temp = pd.merge(cate_temp, lst_2, how = "inner", on=["electrode1", "electrode2",'subject','amp1','freq'])
    else:
        cate_temp = cate[(cate['amp1'] == 2.0)].reset_index(drop=True)
        
        
    for label in range(2):
        df_temp = cate_temp[cate_temp.stim_class == label_list[label]].reset_index(drop=True)
        double = []
        single = []
        for i in range(len(df_temp)):
            df_1 = cate[(cate['electrode1'] == df_temp.electrode1[i]) & 
             (cate['electrode2'] == str()) & 
             (cate['amp1'] == df_temp.amp1[i]) & 
             (cate['freq'] == df_temp.freq[i]) &
             (cate['subject'] == df_temp.subject[i])]
            df_2 = cate[(cate['electrode1'] == df_temp.electrode2[i]) & 
             (cate['electrode2'] == str()) & 
             (cate['amp1'] == df_temp.amp1[i]) & 
             (cate['freq'] == df_temp.freq[i]) &
             (cate['subject'] == df_temp.subject[i])]
            if df_1.size != 0 and df_2.size != 0:
                double_size = sum(df_temp['minor_axis_length'][i])
                single_size1 = sum(df_1['minor_axis_length'].tolist()[0])
                single_size2 = sum(df_2['minor_axis_length'].tolist()[0])
                if double_size>0 and single_size1>0 and single_size2>0:
                    double.append(double_size)
                    combined_image = df_1.avg_image.reset_index(drop=True)[0] + df_2.avg_image.reset_index(drop=True)[0]
                    label_img = skimage.measure.label(combined_image)
                    regions = regionprops(label_img)
                    props = regionprops_table(label_img, properties=('centroid',
                                                                     'orientation',
                                                                     'major_axis_length',
                                                                     'minor_axis_length',
                                                                         'area'))
                    df = pd.DataFrame(props).astype('object')
                    if not df.empty:
                        df.at[0,'centroid-0'] = df.iloc[:, 0].tolist()  # store centroid-x
                        df.at[0,'centroid-1'] = df.iloc[:, 1].tolist()  # store centroid-y
                        df.at[0,'orientation'] = df.iloc[:, 2].tolist()  # store orientation
                        df.at[0,'major_axis_length'] = df.iloc[:, 3].tolist()  # major
                        df.at[0,'minor_axis_length'] = df.iloc[:, 4].tolist()  # minor
                        df.at[0,'area'] = df.iloc[:, 5].tolist()  # area
                    
                    single.append(sum(df['minor_axis_length'][0]))
        lst = []
        for i in range(len(double)):
            if double[i] > 100 or single[i] > 100:
                lst.append(i)
        for i in range(len(lst)):
            del double[lst[-i-1]]
            del single[lst[-i-1]]   
        scatter_correlation(double,single, ax=axes[condition][label])
        scatter_correlation(double,single, ax = axes[condition][2], marker = marker_lst[label], color = color_lst[label], text = False)

axes[0][0].set(xlabel='double', ylabel='single', xlim = (0, 70), ylim = (0,120),  title='minor_axis_length (simultaneous)')
axes[0][1].set(xlabel='double', ylabel='single' , xlim = (0, 70), ylim = (0,120), title='minor_axis_length (sequential)')
axes[0][2].set(xlabel='double', ylabel='single', xlim = (0, 70), ylim = (0,120))
axes[1][2].set(xlabel='double', ylabel='single' , xlim = (0, 70), ylim = (0,120))
axes[1][0].set(xlabel='double', ylabel='single' , xlim = (0, 70), ylim = (0,120), title='minor_axis_length (simultaneous - groups with sequential data only)')
axes[1][1].set(xlabel='double', ylabel='single',  xlim = (0, 70), ylim = (0,120), title='minor_axis_length (simultaneous - groups with simultaneous data only)')